In [1]:
import stats as st
import numpy as np

import plotly.graph_objects as go

### Read data

In [2]:
import importlib

importlib.reload(st)

gaps_cp_df = st.parse_champsim_output(f"../results/GAPS/cp-data")
gaps_wp_df = st.parse_champsim_output(f"../results/GAPS/wp-data")
gaps_wpa_df = st.parse_champsim_output(f"../results/GAPS/wpa-data")

lcf_cp_df = st.parse_champsim_output(f"../results/LCF/cp-data")
lcf_wp_df = st.parse_champsim_output(f"../results/LCF/wp-data")
lcf_wpa_df = st.parse_champsim_output(f"../results/LCF/wpa-data")

spec_cp_df = st.merge_simpoints(st.parse_champsim_output(f"../results/SPEC/cp-data"))
spec_wp_df = st.merge_simpoints(st.parse_champsim_output(f"../results/SPEC/wp-data"))
spec_wpa_df = st.merge_simpoints(st.parse_champsim_output(f"../results/SPEC/wpa-data"))

### Group data

In [3]:
groupings = {
    # Default config
    "default": "default",
    # L1I Prefetchers
    "l1i-next_line": "l1i-next_line",
    "l1i-barsa": "l1i-barsa",
    "l1i-bip": "l1i-bip",
    "l1i-djolt": "l1i-djolt",
    "l1i-epi": "l1i-epi",
    "l1i-fnlmma": "l1i-fnlmma_new",
    "l1i-mana": "l1i-mana",
    "l1i-pips": "l1i-pips",
    "l1i-tap": "l1i-tap",
    # L1D Prefetchers
    "l1d-next_line": "l1d-next_line",
    "l1d-ip_stride": "l1d-ip_stride",
    "l1d-berti": "l1d-berti",
    "l1d-ipcp": "l1d-ipcp",
    # L2C Prefetchers
    "l2c-next_line": "l2c-next_line",
    "l2c-ip_stride": "l2c-ip_stride",
    "l2c-spp": "l2c-spp_dev",
    "l2c-ampm": "l2c-ampm",
    "l2c-bingo": "l2c-bingo",
    "l2c-bop": "l2c-bop",
    "l2c-dspatch": "l2c-dspatch",
    "l2c-ipcp": "l2c-ipcp",
    "l2c-mlop": "l2c-mlop",
    "l2c-sandbox": "l2c-sandbox",
    "l2c-scooby": "l2c-scooby",
    "l2c-sms": "l2c-sms",
    "l2c-spp_ppf": "l2c-spp_ppf_dev",
    "l2c-streamer": "l2c-streamer",
    # LLC Prefetchers
    "llc-next_line": "llc-next_line",
    "llc-ip_stride": "llc-ip_stride",
    # LLC Replacement Policies
    "llc-ship": "ship",
    "llc-srrip": "srrip",
    "llc-drrip": "drrip",
    "llc-mockingjay": "mockingjay",
}

datasets = {
    "gaps": {"cp": gaps_cp_df, "wp": gaps_wp_df, "wpa": gaps_wpa_df},
    "lcf": {"cp": lcf_cp_df, "wp": lcf_wp_df, "wpa": lcf_wpa_df},
    "spec": {"cp": spec_cp_df, "wp": spec_wp_df, "wpa": spec_wpa_df},
}

results = {}
importlib.reload(st)
for label, group_by in groupings.items():
    for category, versions in datasets.items():
        for version, df in versions.items():
            key = f"{category}_{version}_{label}_df"
            grouped = st.group_by(df, group_by)
            results[key] = st.calculate_means(grouped)

In [4]:
big_size = 15
small_size = 30

### gem5 Data

In [ ]:
from collections import defaultdict

gem5_numCycles = {
    "505.mcf_r_00": 131765834,
    "505.mcf_r_01": 190568681,
    "505.mcf_r_02": 258479908,
    "505.mcf_r_03": 561849205,
    "505.mcf_r_04": 170754884,
    "505.mcf_r_05": 117807281,
    "505.mcf_r_06": 227638207,
    "525.x264_r_00": 28896846,
    "525.x264_r_01": 25026959,
    "525.x264_r_02": 14946857,
    "525.x264_r_03": 15848259,
    "525.x264_r_04": 22699256,
    "525.x264_r_05": 24832965,
    "525.x264_r_06": 31744424,
    "531.deepsjeng_r_00": 33699638,
    "531.deepsjeng_r_01": 32487541,
    "531.deepsjeng_r_02": 33466057,
    "531.deepsjeng_r_03": 34235198,
    "531.deepsjeng_r_04": 31449573,
    "531.deepsjeng_r_05": 32821197,
    "531.deepsjeng_r_06": 34477765,
    "531.deepsjeng_r_07": 156984509,
    "531.deepsjeng_r_08": 32014250,
    "541.leela_r_00": 29748683,
    "541.leela_r_01": 48895193,
    "541.leela_r_02": 47727442,
    "541.leela_r_03": 36588421,
    "541.leela_r_04": 49115283,
    "541.leela_r_05": 50208982,
    "541.leela_r_06": 71040246,
    "541.leela_r_07": 49002413,
    "548.exchange2_r_00": 20778577,
    "548.exchange2_r_01": 20315630,
    "548.exchange2_r_02": 21004662,
    "548.exchange2_r_03": 20874075,
    "548.exchange2_r_04": 19708360,
    "548.exchange2_r_05": 18751310,
    "548.exchange2_r_06": 17693791,
    "557.xz_r_00": 64457814,
    "557.xz_r_01": 39773024,
    "557.xz_r_02": 97795435,
    "557.xz_r_03": 66734717,
    "557.xz_r_04": 94616183,
}

gem5_hostSeconds = {
    "505.mcf_r_00": 2600.05,
    "505.mcf_r_01": 2100.27,
    "505.mcf_r_02": 2562.19,
    "505.mcf_r_03": 2869.61,
    "505.mcf_r_04": 1729.60,
    "505.mcf_r_05": 2022.80,
    "505.mcf_r_06": 2565.87,
    "525.x264_r_00": 733.38,
    "525.x264_r_01": 712.83,
    "525.x264_r_02": 552.05,
    "525.x264_r_03": 561.97,
    "525.x264_r_04": 671.50,
    "525.x264_r_05": 678.28,
    "525.x264_r_06": 788.27,
    "531.deepsjeng_r_00": 1020.88,
    "531.deepsjeng_r_01": 1003.83,
    "531.deepsjeng_r_02": 998.93,
    "531.deepsjeng_r_03": 1014.05,
    "531.deepsjeng_r_04": 999.86,
    "531.deepsjeng_r_05": 1010.73,
    "531.deepsjeng_r_06": 1042.04,
    "531.deepsjeng_r_07": 1364.50,
    "531.deepsjeng_r_08": 946.07,
    "541.leela_r_00": 623.50,
    "541.leela_r_01": 1189.46,
    "541.leela_r_02": 1122.56,
    "541.leela_r_03": 970.78,
    "541.leela_r_04": 1183.50,
    "541.leela_r_05": 1222.26,
    "541.leela_r_06": 1315.36,
    "541.leela_r_07": 1231.60,
    "548.exchange2_r_00": 722.09,
    "548.exchange2_r_01": 719.44,
    "548.exchange2_r_02": 678.25,
    "548.exchange2_r_03": 690.88,
    "548.exchange2_r_04": 733.86,
    "548.exchange2_r_05": 715.85,
    "548.exchange2_r_06": 686.67,
    "557.xz_r_00": 1410.61,
    "557.xz_r_01": 615.54,
    "557.xz_r_02": 1522.74,
    "557.xz_r_03": 1287.61,
    "557.xz_r_04": 1454.20,
}

gem5_dcache_overallHits = {
    "505.mcf_r_00": 47840442,
    "505.mcf_r_01": 54794241,
    "505.mcf_r_02": 55151555,
    "505.mcf_r_03": 31662760,
    "505.mcf_r_04": 19389528,
    "505.mcf_r_05": 41795146,
    "505.mcf_r_06": 43139172,
    "525.x264_r_00": 26025484,
    "525.x264_r_01": 27176597,
    "525.x264_r_02": 25085332,
    "525.x264_r_03": 24817260,
    "525.x264_r_04": 25620107,
    "525.x264_r_05": 25540527,
    "525.x264_r_06": 26786239,
    "531.deepsjeng_r_00": 29460816,
    "531.deepsjeng_r_01": 31046415,
    "531.deepsjeng_r_02": 28249969,
    "531.deepsjeng_r_03": 31621500,
    "531.deepsjeng_r_04": 32118445,
    "531.deepsjeng_r_05": 30637204,
    "531.deepsjeng_r_06": 29874052,
    "531.deepsjeng_r_07": 14770449,
    "531.deepsjeng_r_08": 29294976,
    "541.leela_r_00": 20148006,
    "541.leela_r_01": 33160276,
    "541.leela_r_02": 32247212,
    "541.leela_r_03": 31458687,
    "541.leela_r_04": 33974400,
    "541.leela_r_05": 34152219,
    "541.leela_r_06": 32559082,
    "541.leela_r_07": 33921162,
    "548.exchange2_r_00": 26460640,
    "548.exchange2_r_01": 27337921,
    "548.exchange2_r_02": 22876699,
    "548.exchange2_r_03": 24492545,
    "548.exchange2_r_04": 27462406,
    "548.exchange2_r_05": 27699264,
    "548.exchange2_r_06": 27799240,
    "557.xz_r_00": 17257194,
    "557.xz_r_01": 12002151,
    "557.xz_r_02": 29589017,
    "557.xz_r_03": 28425792,
    "557.xz_r_04": 27911030,
}

gem5_dcache_overallMisses = {
    "505.mcf_r_00": 5139468,
    "505.mcf_r_01": 9656422,
    "505.mcf_r_02": 14276899,
    "505.mcf_r_03": 23161600,
    "505.mcf_r_04": 20072499,
    "505.mcf_r_05": 8375716,
    "505.mcf_r_06": 18977249,
    "525.x264_r_00": 2007840,
    "525.x264_r_01": 927720,
    "525.x264_r_02": 73383,
    "525.x264_r_03": 85847,
    "525.x264_r_04": 432772,
    "525.x264_r_05": 1561613,
    "525.x264_r_06": 1892590,
    "531.deepsjeng_r_00": 187072,
    "531.deepsjeng_r_01": 185378,
    "531.deepsjeng_r_02": 207352,
    "531.deepsjeng_r_03": 218898,
    "531.deepsjeng_r_04": 202799,
    "531.deepsjeng_r_05": 203774,
    "531.deepsjeng_r_06": 224970,
    "531.deepsjeng_r_07": 11379168,
    "531.deepsjeng_r_08": 198143,
    "541.leela_r_00": 5528,
    "541.leela_r_01": 115860,
    "541.leela_r_02": 166677,
    "541.leela_r_03": 52611,
    "541.leela_r_04": 257528,
    "541.leela_r_05": 312128,
    "541.leela_r_06": 5094537,
    "541.leela_r_07": 330639,
    "548.exchange2_r_00": 69,
    "548.exchange2_r_01": 11,
    "548.exchange2_r_02": 22,
    "548.exchange2_r_03": 505,
    "548.exchange2_r_04": 443,
    "548.exchange2_r_05": 427,
    "548.exchange2_r_06": 111,
    "557.xz_r_00": 275598,
    "557.xz_r_01": 820044,
    "557.xz_r_02": 1616247,
    "557.xz_r_03": 1094954,
    "557.xz_r_04": 1646223,
}

gem5_icache_overallHits = {
    "505.mcf_r_00": 60396602,
    "505.mcf_r_01": 28801534,
    "505.mcf_r_02": 35614604,
    "505.mcf_r_03": 26484306,
    "505.mcf_r_04": 24735998,
    "505.mcf_r_05": 52853869,
    "505.mcf_r_06": 47573142,
    "525.x264_r_00": 9491887,
    "525.x264_r_01": 11526038,
    "525.x264_r_02": 7415348,
    "525.x264_r_03": 6602864,
    "525.x264_r_04": 11760583,
    "525.x264_r_05": 7522939,
    "525.x264_r_06": 10099100,
    "531.deepsjeng_r_00": 22488496,
    "531.deepsjeng_r_01": 21750251,
    "531.deepsjeng_r_02": 21058879,
    "531.deepsjeng_r_03": 22240447,
    "531.deepsjeng_r_04": 20783850,
    "531.deepsjeng_r_05": 21404617,
    "531.deepsjeng_r_06": 22269622,
    "531.deepsjeng_r_07": 11321539,
    "531.deepsjeng_r_08": 20881387,
    "541.leela_r_00": 18205051,
    "541.leela_r_01": 27753675,
    "541.leela_r_02": 26541423,
    "541.leela_r_03": 22137414,
    "541.leela_r_04": 28910246,
    "541.leela_r_05": 29024323,
    "541.leela_r_06": 27900515,
    "541.leela_r_07": 28757732,
    "548.exchange2_r_00": 10786816,
    "548.exchange2_r_01": 10616915,
    "548.exchange2_r_02": 11593024,
    "548.exchange2_r_03": 11219718,
    "548.exchange2_r_04": 10042563,
    "548.exchange2_r_05": 9773050,
    "548.exchange2_r_06": 9304082,
    "557.xz_r_00": 42402175,
    "557.xz_r_01": 8318437,
    "557.xz_r_02": 35467347,
    "557.xz_r_03": 24654318,
    "557.xz_r_04": 34664561,
}

gem5_icache_overallMisses = {
    "505.mcf_r_00": 0.0,
    "505.mcf_r_01": 0.0,
    "505.mcf_r_02": 0.0,
    "505.mcf_r_03": 0.0,
    "505.mcf_r_04": 0.0,
    "505.mcf_r_05": 0.0,
    "505.mcf_r_06": 3.0,
    "525.x264_r_00": 99823.0,
    "525.x264_r_01": 122576.0,
    "525.x264_r_02": 96.0,
    "525.x264_r_03": 109.0,
    "525.x264_r_04": 224009.0,
    "525.x264_r_05": 17064.0,
    "525.x264_r_06": 95912.0,
    "531.deepsjeng_r_00": 50083.0,
    "531.deepsjeng_r_01": 36479.0,
    "531.deepsjeng_r_02": 94131.0,
    "531.deepsjeng_r_03": 55728.0,
    "531.deepsjeng_r_04": 47450.0,
    "531.deepsjeng_r_05": 68518.0,
    "531.deepsjeng_r_06": 57252.0,
    "531.deepsjeng_r_07": 25541.0,
    "531.deepsjeng_r_08": 61810.0,
    "541.leela_r_00": 5.0,
    "541.leela_r_01": 401.0,
    "541.leela_r_02": 603.0,
    "541.leela_r_03": 11662.0,
    "541.leela_r_04": 204.0,
    "541.leela_r_05": 443.0,
    "541.leela_r_06": 9446.0,
    "541.leela_r_07": 54.0,
    "548.exchange2_r_00": 13721.0,
    "548.exchange2_r_01": 4947.0,
    "548.exchange2_r_02": 28027.0,
    "548.exchange2_r_03": 30488.0,
    "548.exchange2_r_04": 2821.0,
    "548.exchange2_r_05": 1669.0,
    "548.exchange2_r_06": 0.0,
    "557.xz_r_00": 16.0,
    "557.xz_r_01": 0.0,
    "557.xz_r_02": 210.0,
    "557.xz_r_03": 133.0,
    "557.xz_r_04": 23.0,
}

gem5_l2cache_overallHits = {
    "505.mcf_r_00": 1587441.0,
    "505.mcf_r_01": 117694.0,
    "505.mcf_r_02": 139069.0,
    "505.mcf_r_03": 44307.0,
    "505.mcf_r_04": 5836201.0,
    "505.mcf_r_05": 2440432.0,
    "505.mcf_r_06": 5168920.0,
    "525.x264_r_00": 89803.0,
    "525.x264_r_01": 123838.0,
    "525.x264_r_02": 615.0,
    "525.x264_r_03": 1095.0,
    "525.x264_r_04": 163759.0,
    "525.x264_r_05": 22931.0,
    "525.x264_r_06": 92426.0,
    "531.deepsjeng_r_00": 48946.0,
    "531.deepsjeng_r_01": 43468.0,
    "531.deepsjeng_r_02": 77089.0,
    "531.deepsjeng_r_03": 56809.0,
    "531.deepsjeng_r_04": 48880.0,
    "531.deepsjeng_r_05": 58986.0,
    "531.deepsjeng_r_06": 59746.0,
    "531.deepsjeng_r_07": 27644.0,
    "531.deepsjeng_r_08": 60098.0,
    "541.leela_r_00": 2.0,
    "541.leela_r_01": 55806.0,
    "541.leela_r_02": 63177.0,
    "541.leela_r_03": 27383.0,
    "541.leela_r_04": 117307.0,
    "541.leela_r_05": 128826.0,
    "541.leela_r_06": 117723.0,
    "541.leela_r_07": 128121.0,
    "548.exchange2_r_00": 5018.0,
    "548.exchange2_r_01": 1920.0,
    "548.exchange2_r_02": 10216.0,
    "548.exchange2_r_03": 10935.0,
    "548.exchange2_r_04": 625.0,
    "548.exchange2_r_05": 146.0,
    "548.exchange2_r_06": 0.0,
    "557.xz_r_00": 10629.0,
    "557.xz_r_01": 0.0,
    "557.xz_r_02": 233191.0,
    "557.xz_r_03": 235980.0,
    "557.xz_r_04": 344597.0,
}

gem5_l2cache_overallMisses = {
    "505.mcf_r_00": 2025920.0,
    "505.mcf_r_01": 4033375.0,
    "505.mcf_r_02": 6714158.0,
    "505.mcf_r_03": 14521917.0,
    "505.mcf_r_04": 5048971.0,
    "505.mcf_r_05": 1254152.0,
    "505.mcf_r_06": 5454165.0,
    "525.x264_r_00": 170272.0,
    "525.x264_r_01": 69253.0,
    "525.x264_r_02": 18347.0,
    "525.x264_r_03": 24073.0,
    "525.x264_r_04": 32629.0,
    "525.x264_r_05": 145645.0,
    "525.x264_r_06": 176958.0,
    "531.deepsjeng_r_00": 25121.0,
    "531.deepsjeng_r_01": 21774.0,
    "531.deepsjeng_r_02": 27410.0,
    "531.deepsjeng_r_03": 25985.0,
    "531.deepsjeng_r_04": 23938.0,
    "531.deepsjeng_r_05": 25254.0,
    "531.deepsjeng_r_06": 27968.0,
    "531.deepsjeng_r_07": 16864.0,
    "531.deepsjeng_r_08": 25432.0,
    "541.leela_r_00": 3801.0,
    "541.leela_r_01": 2494.0,
    "541.leela_r_02": 1685.0,
    "541.leela_r_03": 1257.0,
    "541.leela_r_04": 2260.0,
    "541.leela_r_05": 1862.0,
    "541.leela_r_06": 795587.0,
    "541.leela_r_07": 1267.0,
    "548.exchange2_r_00": 53.0,
    "548.exchange2_r_01": 14.0,
    "548.exchange2_r_02": 47.0,
    "548.exchange2_r_03": 906.0,
    "548.exchange2_r_04": 846.0,
    "548.exchange2_r_05": 844.0,
    "548.exchange2_r_06": 34.0,
    "557.xz_r_00": 45907.0,
    "557.xz_r_01": 45558.0,
    "557.xz_r_02": 621667.0,
    "557.xz_r_03": 268951.0,
    "557.xz_r_04": 589396.0,
}

weights = {
    "505.mcf_r_00": 0.073125,
    "505.mcf_r_01": 0.167787,
    "505.mcf_r_02": 0.053478,
    "505.mcf_r_03": 0.034881,
    "505.mcf_r_04": 0.095293,
    "505.mcf_r_05": 0.553688,
    "505.mcf_r_06": 0.021748,
    "525.x264_r_00": 0.051166,
    "525.x264_r_01": 0.107270,
    "525.x264_r_02": 0.205452,
    "525.x264_r_03": 0.220269,
    "525.x264_r_04": 0.268866,
    "525.x264_r_05": 0.040893,
    "525.x264_r_06": 0.106085,
    "531.deepsjeng_r_00": 0.095342,
    "531.deepsjeng_r_01": 0.144630,
    "531.deepsjeng_r_02": 0.088145,
    "531.deepsjeng_r_03": 0.133521,
    "531.deepsjeng_r_04": 0.082408,
    "531.deepsjeng_r_05": 0.067856,
    "531.deepsjeng_r_06": 0.194701,
    "531.deepsjeng_r_07": 0.000626,
    "531.deepsjeng_r_08": 0.192771,
    "541.leela_r_00": 0.000391,
    "541.leela_r_01": 0.157962,
    "541.leela_r_02": 0.124884,
    "541.leela_r_03": 0.022524,
    "541.leela_r_04": 0.192065,
    "541.leela_r_05": 0.251185,
    "541.leela_r_06": 0.024527,
    "541.leela_r_07": 0.226462,
    "548.exchange2_r_00": 0.084607,
    "548.exchange2_r_01": 0.165750,
    "548.exchange2_r_02": 0.021238,
    "548.exchange2_r_03": 0.040095,
    "548.exchange2_r_04": 0.119466,
    "548.exchange2_r_05": 0.307961,
    "548.exchange2_r_06": 0.260883,
    "557.xz_r_00": 0.116841,
    "557.xz_r_01": 0.046737,
    "557.xz_r_02": 0.351867,
    "557.xz_r_03": 0.004298,
    "557.xz_r_04": 0.480258,
}

gem5_ipc = {k: 100_000_000 / v for k, v in gem5_numCycles.items()}


# Function to calculate weighted averages
def calculate_weighted_average(data_dict, weights_dict):
    benchmark_weighted_sum = defaultdict(float)
    benchmark_total_weight = defaultdict(float)
    result_dict = {}

    for key, weight in weights_dict.items():
        value = data_dict.get(key)
        if value is None:
            # Try with alternative key if the original key is not found
            alt_key = key.replace("_r_", "_")
            value = data_dict.get(alt_key)

        if value is not None:
            prefix = key.split("_r_")[0]
            benchmark_weighted_sum[prefix] += value * weight
            benchmark_total_weight[prefix] += weight
        else:
            print(f"Missing data for: {key}")

    for benchmark in sorted(benchmark_weighted_sum):
        total_sum = benchmark_weighted_sum[benchmark]
        total_weight = benchmark_total_weight[benchmark]
        weighted_avg = total_sum / total_weight if total_weight != 0 else 0
        result_dict[benchmark] = weighted_avg
    return result_dict


# Calculate weighted averages for each metric
gem5_wipc = calculate_weighted_average(gem5_ipc, weights)
gem5_wtime = calculate_weighted_average(gem5_hostSeconds, weights)
gem5_wdcache_hits = calculate_weighted_average(gem5_dcache_overallHits, weights)
gem5_wdcache_misses = calculate_weighted_average(gem5_dcache_overallMisses, weights)
gem5_wicache_hits = calculate_weighted_average(gem5_icache_overallHits, weights)
gem5_wicache_misses = calculate_weighted_average(gem5_icache_overallMisses, weights)
gem5_wl2cache_hits = calculate_weighted_average(gem5_l2cache_overallHits, weights)
gem5_wl2cache_misses = calculate_weighted_average(gem5_l2cache_overallMisses, weights)

### IPC gem5 v ChampSim No-WP v ChampSim ED-WP

In [ ]:
suites = ["lcf", "gaps", "spec"]
gem5_label = "gem5"
cp_label = "No-WP"
wp_label = "ED-WP"
colors = {"gem5": "black", "ED-WP": "#1f77b4", "No-WP": "#ff7f0e"}

all_benchmarks = [
    "web-search",
    "media-stream",
    "specjbb",
    "wikipedia",
    # "xapian",
    "finagle-http",
    "speedometer2.0",
    "data-serving",
    "kafka",
    "tpcc",
    "cassandra",
    "finagle-chirper",
    "verilator-bolted",
    "tomcat",
    "sssp",
    "pr",
    "prspmv",
    "cc",
    "bc",
    "ccsv",
    "bfs",
    "tc",
    "548.exchange2",
    "525.x264",
    "531.deepsjeng",
    "557.xz",
    "541.leela",
    "505.mcf",
]
import pandas as pd

# Add the LCF and GAP benchmarks
gem5_wipc["web-search"] = 100_000_000 / 16473707
gem5_wipc["media-stream"] = 100_000_000 / 95746267
gem5_wipc["specjbb"] = 100_000_000 / 266164899
gem5_wipc["wikipedia"] = 100_000_000 / 77447466
gem5_wipc["xapian"] = 0
gem5_wipc["finagle-http"] = 100_000_000 / 81383219
gem5_wipc["speedometer2.0"] = 100_000_000 / 47488791
gem5_wipc["data-serving"] = 100_000_000 / 103769915
gem5_wipc["kafka"] = 100_000_000 / 79159920
gem5_wipc["tpcc"] = 100_000_000 / 61439096
gem5_wipc["cassandra"] = 100_000_000 / 97091994
gem5_wipc["finagle-chirper"] = 100_000_000 / 104312048
gem5_wipc["verilator-bolted"] = 100_000_000 / 81672355
gem5_wipc["tomcat"] = 100_000_000 / 116482431
gem5_wipc["sssp"] = 100_000_000 / 244354558
gem5_wipc["pr"] = 100_000_000 / 476048849
gem5_wipc["prspmv"] = 100_000_000 / 485240310
gem5_wipc["cc"] = 100_000_000 / 780467232
gem5_wipc["bc"] = 100_000_000 / 797901088
gem5_wipc["ccsv"] = 100_000_000 / 319483518
gem5_wipc["bfs"] = 100_000_000 / 287132347
gem5_wipc["tc"] = 100_000_000 / 155976750
cp_ipc = pd.concat(
    [
        results["lcf_cp_default_df"]["IPC"],
        results["gaps_cp_default_df"]["IPC"],
        results["spec_cp_default_df"]["IPC"],
    ]
)
wp_ipc = pd.concat(
    [
        results["lcf_wp_default_df"]["IPC"],
        results["gaps_wp_default_df"]["IPC"],
        results["spec_wp_default_df"]["IPC"],
    ]
)

# Drop amean and gmean rows
cp_ipc = cp_ipc[~cp_ipc.index.isin(["amean", "gmean"])]
wp_ipc = wp_ipc[~wp_ipc.index.isin(["amean", "gmean"])]

# Convert gem5_wipc to a pandas Series for consistency
gem5_wipc = pd.Series(gem5_wipc)

# Sort the IPC values to match the order of all_benchmarks
gem5_wipc = gem5_wipc.reindex(all_benchmarks).fillna(0)
# Ensure all IPC series are aligned with all_benchmarks
cp_ipc = cp_ipc.reindex(all_benchmarks).fillna(0)
wp_ipc = wp_ipc.reindex(all_benchmarks).fillna(0)

fig = go.Figure()

fig.add_trace(
    go.Bar(x=all_benchmarks, y=gem5_wipc, name=gem5_label, marker_color=colors["gem5"])
)

fig.add_trace(
    go.Bar(x=all_benchmarks, y=cp_ipc, name=cp_label, marker_color=colors["No-WP"])
)

fig.add_trace(
    go.Bar(x=all_benchmarks, y=wp_ipc, name=wp_label, marker_color=colors["ED-WP"])
)

# Add vertical lines to separate suites
fig.add_vline(x=12.5, line_width=2, line_dash="dash", line_color="gray")
fig.add_vline(x=20.5, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for label in ["LCF", "GAP", "SPEC"]:
    fig.add_annotation(
        x=(
            12.5 / 2
            if label == "LCF"
            else 12.5 + (20.5 - 12.5) / 2 if label == "GAP" else 24
        ),
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    # title="WP and WPA Speedup vs CP (Default)",
    xaxis=dict(
        # title=dict(
        #     text="Benchmark",
        #     standoff=50,  # Add vertical space between axis and title
        # ),
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,  # Ensure horizontal grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    yaxis=dict(
        title="IPC",
        showgrid=True,  # Ensure vertical grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    barmode="group",
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
    font=dict(family="Arial, sans-serif", size=big_size),  # Arial is consistent
)

fig.show()
fig.write_image("pdf/gem5_ipc_comparison.pdf", width=1100, height=400, scale=2)

### Simulation Overhead

In [ ]:
sim_time_gaps_cp = results["gaps_cp_default_df"]["Simulation Time"]["amean"]
sim_time_gaps_wp = results["gaps_wp_default_df"]["Simulation Time"]["amean"]
sim_time_gaps_gem5 = {
    "sssp": 2430.43,
    "pr": 4143.26,
    "prspmv": 4761.63,
    "cc": 4288.41,
    "bc": 4386.38,
    "ccsv": 5093.96,
    "bfs": 3731.67,
    "tc": 2923.82,
}

sim_time_gaps_gem5 = sum(sim_time_gaps_gem5.values()) / len(sim_time_gaps_gem5.values())

gaps_sim_overhead = ((sim_time_gaps_wp - sim_time_gaps_cp) / sim_time_gaps_cp) * 100
gaps_sim_overhead_gem5 = (
    (sim_time_gaps_gem5 - sim_time_gaps_cp) / sim_time_gaps_cp
) * 100

sim_time_lcf_cp = results["lcf_cp_default_df"]["Simulation Time"]["amean"]
sim_time_lcf_wp = results["lcf_wp_default_df"]["Simulation Time"]["amean"]
sim_time_lcf_gem5 = {
    "web-search": 1066.56,
    "media-stream": 1832.95,
    "specjbb": 2809.91,
    "wikipedia": 1809.64,
    # "xapian": 0.0,
    "finagle-http": 1861.85,
    "speedometer2.0": 1553.5,
    "data-serving": 2144.28,
    "kafka": 1489.56,
    "tpcc": 1745.23,
    "cassandra": 1590.09,
    "finagle-chirper": 2100.05,
    "verilator-bolted": 1582.68,
    "tomcat": 2162.9,
}

sim_time_lcf_gem5 = sum(sim_time_lcf_gem5.values()) / len(sim_time_lcf_gem5.values())

lcf_sim_overhead = ((sim_time_lcf_wp - sim_time_lcf_cp) / sim_time_lcf_cp) * 100
lcf_sim_overhead_gem5 = ((sim_time_lcf_gem5 - sim_time_lcf_cp) / sim_time_lcf_cp) * 100

sim_time_spec_cp = results["spec_cp_default_df"]["Simulation Time"]["amean"]
sim_time_spec_wp = results["spec_wp_default_df"]["Simulation Time"]["amean"]
sim_time_spec_gem5 = {
    "548.exchange2": gem5_wtime["548.exchange2"],
    "525.x264": gem5_wtime["525.x264"],
    "531.deepsjeng": gem5_wtime["531.deepsjeng"],
    "557.xz": gem5_wtime["557.xz"],
    "541.leela": gem5_wtime["541.leela"],
    "505.mcf": gem5_wtime["505.mcf"],
}

spec_sim_overhead = ((sim_time_spec_wp - sim_time_spec_cp) / sim_time_spec_cp) * 100
spec_sim_overhead_gem5 = (
    (
        sum(sim_time_spec_gem5.values()) / len(sim_time_spec_gem5.values())
        - sim_time_spec_cp
    )
    / sim_time_spec_cp
) * 100

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=["LCF", "GAP", "SPEC"],
        y=[lcf_sim_overhead, gaps_sim_overhead, spec_sim_overhead],
        name="ChampSim ED-WP",
        marker_color="indianred",
    )
)
fig.add_trace(
    go.Bar(
        x=["LCF", "GAP", "SPEC"],
        y=[lcf_sim_overhead_gem5, gaps_sim_overhead_gem5, spec_sim_overhead_gem5],
        name="gem5",
        marker_color="darkorange",
    )
)
fig.update_layout(
    # title="Simulation Overhead of GAPS, LCF, and SPEC",
    # xaxis_title="Benchmark Suite",
    yaxis_title="Simulation Overhead (%)",
    template="plotly_white",
    font=dict(size=small_size, family="Arial, sans-serif"),
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="center",
        x=0.5,
        font=dict(size=small_size, family="Arial, sans-serif"),
        # title_text="Simulation Method",
    ),
    yaxis=dict(
        range=[0, 60],
    ),
)
fig.update_xaxes(tickfont=dict(size=small_size, family="Arial, sans-serif"))
fig.update_yaxes(tickfont=dict(size=small_size, family="Arial, sans-serif"))
fig.write_image("pdf/simulation_overhead.pdf", width=1100, height=600, scale=2)
fig.show()

### Cache gem5 v ChampSim

#### Data

In [ ]:
# L1D Hit
gem5_wdcache_hits["web-search"] = 35968904
gem5_wdcache_hits["media-stream"] = 38621066
gem5_wdcache_hits["specjbb"] = 12923895
gem5_wdcache_hits["wikipedia"] = 38854026
gem5_wdcache_hits["xapian"] = 1
gem5_wdcache_hits["finagle-http"] = 33361391
gem5_wdcache_hits["speedometer2.0"] = 39292059
gem5_wdcache_hits["data-serving"] = 37614374
gem5_wdcache_hits["kafka"] = 37734261
gem5_wdcache_hits["tpcc"] = 40174212
gem5_wdcache_hits["cassandra"] = 39353762
gem5_wdcache_hits["finagle-chirper"] = 35707018
gem5_wdcache_hits["verilator-bolted"] = 40994767
gem5_wdcache_hits["tomcat"] = 40762780
gem5_wdcache_hits["sssp"] = 36257276
gem5_wdcache_hits["pr"] = 14359301
gem5_wdcache_hits["prspmv"] = 12324550
gem5_wdcache_hits["cc"] = 44627206
gem5_wdcache_hits["bc"] = 42980442
gem5_wdcache_hits["ccsv"] = 83995681
gem5_wdcache_hits["bfs"] = 47787664
gem5_wdcache_hits["tc"] = 30452043

# L1D Miss
gem5_wdcache_misses["web-search"] = 237349
gem5_wdcache_misses["media-stream"] = 14229197
gem5_wdcache_misses["specjbb"] = 17309830
gem5_wdcache_misses["wikipedia"] = 1964956
gem5_wdcache_misses["xapian"] = 1
gem5_wdcache_misses["finagle-http"] = 3885427
gem5_wdcache_misses["speedometer2.0"] = 1182024
gem5_wdcache_misses["data-serving"] = 2199949
gem5_wdcache_misses["kafka"] = 6534359
gem5_wdcache_misses["tpcc"] = 2398564
gem5_wdcache_misses["cassandra"] = 3445860
gem5_wdcache_misses["finagle-chirper"] = 4164746
gem5_wdcache_misses["verilator-bolted"] = 141326
gem5_wdcache_misses["tomcat"] = 2600197
gem5_wdcache_misses["sssp"] = 13767980
gem5_wdcache_misses["pr"] = 28887203
gem5_wdcache_misses["prspmv"] = 30587967
gem5_wdcache_misses["cc"] = 5837938
gem5_wdcache_misses["bc"] = 5808280
gem5_wdcache_misses["ccsv"] = 17728175
gem5_wdcache_misses["bfs"] = 9728277
gem5_wdcache_misses["tc"] = 7229056

# L1I Hit
gem5_wicache_hits["web-search"] = 9127538
gem5_wicache_hits["media-stream"] = 21692267
gem5_wicache_hits["specjbb"] = 7450176
gem5_wicache_hits["wikipedia"] = 24935767
gem5_wicache_hits["xapian"] = 1
gem5_wicache_hits["finagle-http"] = 16919175
gem5_wicache_hits["speedometer2.0"] = 18742811
gem5_wicache_hits["data-serving"] = 29461232
gem5_wicache_hits["kafka"] = 16243888
gem5_wicache_hits["tpcc"] = 23596244
gem5_wicache_hits["cassandra"] = 25275264
gem5_wicache_hits["finagle-chirper"] = 23618541
gem5_wicache_hits["verilator-bolted"] = 9143757
gem5_wicache_hits["tomcat"] = 34300579
gem5_wicache_hits["sssp"] = 25661919
gem5_wicache_hits["pr"] = 3322523
gem5_wicache_hits["prspmv"] = 3678818
gem5_wicache_hits["cc"] = 74079791
gem5_wicache_hits["bc"] = 74368361
gem5_wicache_hits["ccsv"] = 126572542
gem5_wicache_hits["bfs"] = 102179457
gem5_wicache_hits["tc"] = 63786459

# L1I Miss
gem5_wicache_misses["web-search"] = 48104
gem5_wicache_misses["media-stream"] = 2910924
gem5_wicache_misses["specjbb"] = 1336544
gem5_wicache_misses["wikipedia"] = 2347082
gem5_wicache_misses["xapian"] = 1
gem5_wicache_misses["finagle-http"] = 5466505
gem5_wicache_misses["speedometer2.0"] = 3092798
gem5_wicache_misses["data-serving"] = 3870821
gem5_wicache_misses["kafka"] = 1323649
gem5_wicache_misses["tpcc"] = 3577342
gem5_wicache_misses["cassandra"] = 718307
gem5_wicache_misses["finagle-chirper"] = 5524906
gem5_wicache_misses["verilator-bolted"] = 7585457
gem5_wicache_misses["tomcat"] = 4500914
gem5_wicache_misses["sssp"] = 11526
gem5_wicache_misses["pr"] = 1726
gem5_wicache_misses["prspmv"] = 1821
gem5_wicache_misses["cc"] = 4768
gem5_wicache_misses["bc"] = 4751
gem5_wicache_misses["ccsv"] = 1181
gem5_wicache_misses["bfs"] = 17897
gem5_wicache_misses["tc"] = 1579

# L2 Hit
gem5_wl2cache_hits["web-search"] = 147189
gem5_wl2cache_hits["media-stream"] = 4096232
gem5_wl2cache_hits["specjbb"] = 636851
gem5_wl2cache_hits["wikipedia"] = 2828752
gem5_wl2cache_hits["xapian"] = 1
gem5_wl2cache_hits["finagle-http"] = 5939952
gem5_wl2cache_hits["speedometer2.0"] = 3114284
gem5_wl2cache_hits["data-serving"] = 4220212
gem5_wl2cache_hits["kafka"] = 3036019
gem5_wl2cache_hits["tpcc"] = 3877220
gem5_wl2cache_hits["cassandra"] = 6294093
gem5_wl2cache_hits["finagle-chirper"] = 4992473
gem5_wl2cache_hits["verilator-bolted"] = 859726
gem5_wl2cache_hits["tomcat"] = 3257548
gem5_wl2cache_hits["sssp"] = 6284035
gem5_wl2cache_hits["pr"] = 13111258
gem5_wl2cache_hits["prspmv"] = 13791146
gem5_wl2cache_hits["cc"] = 1409536
gem5_wl2cache_hits["bc"] = 1478358
gem5_wl2cache_hits["ccsv"] = 7270386
gem5_wl2cache_hits["bfs"] = 325483
gem5_wl2cache_hits["tc"] = 1255

# L2 Miss
gem5_wl2cache_misses["web-search"] = 43202
gem5_wl2cache_misses["media-stream"] = 1562665
gem5_wl2cache_misses["specjbb"] = 1857143
gem5_wl2cache_misses["wikipedia"] = 397256
gem5_wl2cache_misses["xapian"] = 1
gem5_wl2cache_misses["finagle-http"] = 2245720
gem5_wl2cache_misses["speedometer2.0"] = 306300
gem5_wl2cache_misses["data-serving"] = 1039190
gem5_wl2cache_misses["kafka"] = 748784
gem5_wl2cache_misses["tpcc"] = 458728
gem5_wl2cache_misses["cassandra"] = 181220
gem5_wl2cache_misses["finagle-chirper"] = 3200266
gem5_wl2cache_misses["verilator-bolted"] = 5736406
gem5_wl2cache_misses["tomcat"] = 2066432
gem5_wl2cache_misses["sssp"] = 6612669
gem5_wl2cache_misses["pr"] = 15495254
gem5_wl2cache_misses["prspmv"] = 16300463
gem5_wl2cache_misses["cc"] = 5251557
gem5_wl2cache_misses["bc"] = 5291150
gem5_wl2cache_misses["ccsv"] = 7836347
gem5_wl2cache_misses["bfs"] = 3203581
gem5_wl2cache_misses["tc"] = 564292

# L1D Hit Ratio
gem5_wdcache_hit_ratio = {
    benchmark: hits / (hits + misses)
    for benchmark, hits in gem5_wdcache_hits.items()
    for misses in [gem5_wdcache_misses[benchmark]]
}
# L1D Miss Ratio
gem5_wdcache_miss_ratio = {
    benchmark: misses / (hits + misses)
    for benchmark, misses in gem5_wdcache_misses.items()
    for hits in [gem5_wdcache_hits[benchmark]]
}
# L1D MPKI
gem5_wdcache_mpkis = {
    benchmark: (misses * 1000) / 100_000_000
    for benchmark, misses in gem5_wdcache_misses.items()
}

# L1I Hit Ratio
gem5_wicache_hit_ratio = {
    benchmark: hits / (hits + misses)
    for benchmark, hits in gem5_wicache_hits.items()
    for misses in [gem5_wicache_misses[benchmark]]
}
# L1I Miss Ratio
gem5_wicache_miss_ratio = {
    benchmark: misses / (hits + misses)
    for benchmark, misses in gem5_wicache_misses.items()
    for hits in [gem5_wicache_hits[benchmark]]
}
# L1I MPKI
gem5_wicache_mpkis = {
    benchmark: (misses * 1000) / 100_000_000
    for benchmark, misses in gem5_wicache_misses.items()
}

# L2 Hit Ratio
gem5_wl2cache_hit_ratio = {
    benchmark: hits / (hits + misses)
    for benchmark, hits in gem5_wl2cache_hits.items()
    for misses in [gem5_wl2cache_misses[benchmark]]
}
# L2 Miss Ratio
gem5_wl2cache_miss_ratio = {
    benchmark: misses / (hits + misses)
    for benchmark, misses in gem5_wl2cache_misses.items()
    for hits in [gem5_wl2cache_hits[benchmark]]
}
# L2 MPKI
gem5_wl2cache_mpkis = {
    benchmark: (misses * 1000) / 100_000_000
    for benchmark, misses in gem5_wl2cache_misses.items()
}

#### Plot Code

In [ ]:
import pandas as pd
import plotly.graph_objects as go


def create_cache_plot(
    gem5_data,
    cp_data_col,
    wp_data_col,
    div_data_col,
    yaxis_title,
    all_benchmarks,
    colors,
    gem5_label,
    cp_label,
    wp_label,
):
    """
    Creates a Plotly bar chart for D-cache hits or misses.

    Args:
        gem5_data (dict): Dictionary of gem5_wdcache hits or misses.
        cp_data_col (str): Column name for "No-WP" data (e.g., "L1D_TOTAL_HITS").
        wp_data_col (str): Column name for "ED-WP" data (e.g., "L1D_TOTAL_HITS").
        yaxis_title (str): Title for the Y-axis (e.g., "L1D Hits (PKI)").
        all_benchmarks (list): List of all benchmark names.
        colors (dict): Dictionary mapping labels to colors.
        gem5_label (str): Label for gem5 data.
        cp_label (str): Label for No-WP data.
        wp_label (str): Label for ED-WP data.

    Returns:
        go.Figure: Plotly figure object.
    """
    data_series = pd.Series(gem5_data).reindex(all_benchmarks).fillna(0)

    cp_series = pd.concat(
        [
            results["lcf_cp_default_df"][cp_data_col],
            results["gaps_cp_default_df"][cp_data_col],
            results["spec_cp_default_df"][cp_data_col],
        ]
    )
    # / pd.concat(
    #     [
    #         results["lcf_cp_default_df"][div_data_col],
    #         results["gaps_cp_default_df"][div_data_col],
    #         results["spec_cp_default_df"][div_data_col],
    #     ]
    # )
    wp_series = pd.concat(
        [
            results["lcf_wp_default_df"][wp_data_col],
            results["gaps_wp_default_df"][wp_data_col],
            results["spec_wp_default_df"][wp_data_col],
        ]
    )
    # / pd.concat(
    #     [
    #         results["lcf_wp_default_df"][div_data_col],
    #         results["gaps_wp_default_df"][div_data_col],
    #         results["spec_wp_default_df"][div_data_col],
    #     ]
    # )

    # Drop amean and gmean rows
    cp_series = cp_series[~cp_series.index.isin(["amean", "gmean"])]
    wp_series = wp_series[~wp_series.index.isin(["amean", "gmean"])]

    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=data_series,
            name="gem5",
            marker_color=colors[gem5_label],
        )
    )
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=cp_series,
            name="No-WP",
            marker_color=colors[cp_label],
        )
    )
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=wp_series,
            name="ED-WP",
            marker_color=colors[wp_label],
        )
    )

    fig.update_layout(
        xaxis=dict(
            title=dict(text="Benchmark", standoff=50),
            tickangle=-45,
            tickmode="array",
            tickvals=list(range(len(all_benchmarks))),
            ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
            showgrid=True,
            gridcolor="lightgray",
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor="black",
            zerolinewidth=2,
            griddash="dot",
        ),
        yaxis=dict(
            title=yaxis_title,
            showgrid=True,
            gridcolor="lightgray",
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor="black",
            zerolinewidth=2,
            griddash="dot",
        ),
        barmode="group",
        legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
        plot_bgcolor="white",
        margin=dict(l=20, r=20, t=20, b=20),
    )
    return fig

#### Plots

In [ ]:
fig_dhits = create_cache_plot(
    gem5_data=gem5_wdcache_hits,
    cp_data_col="L1D_TOTAL_HITS",
    wp_data_col="L1D_TOTAL_HITS",
    div_data_col="L1D_TOTAL_ACCESS",
    yaxis_title="L1D Hits",
    all_benchmarks=all_benchmarks,
    colors=colors,
    gem5_label=gem5_label,
    cp_label=cp_label,
    wp_label=wp_label,
)
# fig_dhits.show()

fig_dmisses = create_cache_plot(
    gem5_data=gem5_wdcache_misses,
    cp_data_col="L1D_TOTAL_MISS",
    wp_data_col="L1D_TOTAL_MISS",
    div_data_col="L1D_TOTAL_ACCESS",
    yaxis_title="L1D Misses",
    all_benchmarks=all_benchmarks,
    colors=colors,
    gem5_label=gem5_label,
    cp_label=cp_label,
    wp_label=wp_label,
)
# fig_dmisses.show()

fig_ihits = create_cache_plot(
    gem5_data=gem5_wicache_hits,
    cp_data_col="L1I_TOTAL_HITS",
    wp_data_col="L1I_TOTAL_HITS",
    div_data_col="L1I_TOTAL_ACCESS",
    yaxis_title="L1I Hits",
    all_benchmarks=all_benchmarks,
    colors=colors,
    gem5_label=gem5_label,
    cp_label=cp_label,
    wp_label=wp_label,
)
# fig_ihits.show()

fig_imisses = create_cache_plot(
    gem5_data=gem5_wicache_misses,
    cp_data_col="L1I_TOTAL_MISS",
    wp_data_col="L1I_TOTAL_MISS",
    div_data_col="L1I_TOTAL_ACCESS",
    yaxis_title="L1I Misses",
    all_benchmarks=all_benchmarks,
    colors=colors,
    gem5_label=gem5_label,
    cp_label=cp_label,
    wp_label=wp_label,
)
# fig_imisses.show()

fig_l2hits = create_cache_plot(
    gem5_data=gem5_wl2cache_hits,
    cp_data_col="L2C_TOTAL_HITS",
    wp_data_col="L2C_TOTAL_HITS",
    div_data_col="L2C_TOTAL_ACCESS",
    yaxis_title="L2 Hits",
    all_benchmarks=all_benchmarks,
    colors=colors,
    gem5_label=gem5_label,
    cp_label=cp_label,
    wp_label=wp_label,
)
# fig_l2hits.show()

fig_l2misses = create_cache_plot(
    gem5_data=gem5_wl2cache_misses,
    cp_data_col="L2C_TOTAL_MISS",
    wp_data_col="L2C_TOTAL_MISS",
    div_data_col="L2C_TOTAL_ACCESS",
    yaxis_title="L2 Misses",
    all_benchmarks=all_benchmarks,
    colors=colors,
    gem5_label=gem5_label,
    cp_label=cp_label,
    wp_label=wp_label,
)
# fig_l2misses.show()

# # Save the figures as PDF files
fig_dhits.write_image("pdf/dcache_hits.pdf", width=1100, height=600, scale=2)
fig_dmisses.write_image("pdf/dcache_misses.pdf", width=1100, height=600, scale=2)
fig_ihits.write_image("pdf/icache_hits.pdf", width=1100, height=600, scale=2)
fig_imisses.write_image("pdf/icache_misses.pdf", width=1100, height=600, scale=2)
fig_l2hits.write_image("pdf/l2cache_hits.pdf", width=1100, height=600, scale=2)
fig_l2misses.write_image("pdf/l2cache_misses.pdf", width=1100, height=600, scale=2)

### Instruction Overhead & MPKI

In [ ]:
suites = ["lcf", "gaps", "spec"]
colors = {"ED-WP": "steelblue", "TI-WP": "indianred"}

all_benchmarks = []
wp_speedups = []
wpa_speedups = []
suite_boundaries = []
suite_labels = []
mpki_values_lcf = []
mpki_values_gaps = []
mpki_values_spec = []

x_position = 0

for suite in suites:
    wp = results[f"{suite}_wp_default_df"]["wrong_path_insts_executed"]
    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    wp = wp.drop("gmean", errors="ignore")
    benchmarks = list(wp.index.drop(["gmean", "amean"], errors="ignore"))

    mpki = mpki.drop("gmean", errors="ignore")
    benchmarks = list(mpki.index.drop(["gmean", "amean"], errors="ignore"))

    # Create list of tuples of (benchmark, mpki)
    benchmark_mpki_pairs = list(zip(benchmarks, mpki))

    # Sort the benchmarks by MPKI value (ascending order for MPKI)
    benchmark_mpki_pairs.sort(key=lambda x: x[1])  # Sort ascending by MPKI

    # Extract sorted benchmarks and MPKI values
    sorted_benchmarks = [bm for bm, _ in benchmark_mpki_pairs]
    sorted_mpki_values = [mpki_val for _, mpki_val in benchmark_mpki_pairs]

    # Remove "amean" if it was still in any list
    if "amean" in sorted_benchmarks:
        amean_index = sorted_benchmarks.index("amean")
        sorted_mpki_values.pop(amean_index)  # Remove MPKI value for "amean"
        sorted_benchmarks.pop(amean_index)  # Remove "amean" from benchmark list

    benchmarks_labeled = [f"{suite.upper()}_{bm}" for bm in sorted_benchmarks]
    all_benchmarks.extend(benchmarks_labeled)

    # Update wp_speedups according to sorted benchmarks
    wp_speedups.extend(wp[sorted_benchmarks] / 100_000_000 * 100)

    # Store MPKI values for each suite
    if suite == "lcf":
        mpki_values_lcf.extend(sorted_mpki_values)
    elif suite == "gaps":
        mpki_values_gaps.extend(sorted_mpki_values)
    elif suite == "spec":
        mpki_values_spec.extend(sorted_mpki_values)

    # Save mid-point and boundary for annotations
    suite_labels.append(
        (
            x_position + len(sorted_benchmarks) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(sorted_benchmarks)
    suite_boundaries.append(x_position - 0.5)


fig = go.Figure()

fig.add_trace(
    go.Bar(x=all_benchmarks, y=wp_speedups, name="ED-WP", marker_color=colors["ED-WP"])
)

fig.add_trace(
    go.Scatter(
        x=all_benchmarks[: len(mpki_values_lcf)],
        y=mpki_values_lcf,
        mode="lines+markers",
        name="MPKI LCF",
        line=dict(color="red"),
        yaxis="y2",
    )
)

fig.add_trace(
    go.Scatter(
        x=all_benchmarks[
            len(mpki_values_lcf) : len(mpki_values_lcf) + len(mpki_values_gaps)
        ],
        y=mpki_values_gaps,
        mode="lines+markers",
        name="MPKI GAPS",
        line=dict(color="green"),
        yaxis="y2",
    )
)

fig.add_trace(
    go.Scatter(
        x=all_benchmarks[len(mpki_values_lcf) + len(mpki_values_gaps) :],
        y=mpki_values_spec,
        mode="lines+markers",
        name="MPKI SPEC",
        line=dict(color="blue"),
        yaxis="y2",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite annotations above
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    # title="WP and WPA Speedup vs CP (Default)",
    xaxis=dict(
        # title=dict(
        #     text="Benchmark",
        #     standoff=50,  # Add vertical space between axis and title
        # ),
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,  # Ensure horizontal grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    yaxis=dict(
        title="Extra Instr. (%)",
        # range=[-5, 40],  # max(max(wp_speedups), max(wpa_speedups)) * 1.1
        showgrid=True,  # Ensure vertical grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
        range=[0, 250],  # Adjusted range for better visibility
    ),
    yaxis2=dict(
        title="Branch MPKI",
        overlaying="y",
        side="right",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=False,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dash",
        range=[0, 40],
        dtick=8,
    ),
    barmode="group",
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),  # left, right, top, bottom
    showlegend=False,
    font=dict(family="Arial, sans-serif", size=big_size),  # Arial is consistent
)

fig.show()

# Save the figure as an interactive HTML file
fig.write_html("html/overhead_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("pdf/overhead_plot.pdf", format="pdf", width=1100, height=350, scale=2)

### L1I Hits & Misses

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
hits_diffs = []
misses_diffs = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    # Load data
    cp_hits = results[f"{suite}_cp_default_df"]["L1I_TOTAL_HITS"]
    wp_hits = results[f"{suite}_wp_default_df"]["L1I_TOTAL_HITS"]
    cp_misses = results[f"{suite}_cp_default_df"]["L1I_TOTAL_MISS"]
    wp_misses = results[f"{suite}_wp_default_df"]["L1I_TOTAL_MISS"]
    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean
    for metric in [cp_hits, wp_hits, cp_misses, wp_misses, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Get benchmark list excluding amean
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x])

    # Add suite-prefixed amean at the end if available
    if "amean" in mpki.index:
        benchmarks_sorted.append(f"{suite.upper()}_amean")

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in benchmarks_sorted
    ]

    all_benchmarks.extend(labeled_benchmarks)

    # Get diffs in the same order
    hits_diff = ((wp_hits - cp_hits) / cp_hits) * 100
    misses_diff = ((wp_misses - cp_misses) / cp_misses) * 100

    hits_diffs.extend(
        [
            hits_diff.get(bm if "amean" not in bm else "amean")
            for bm in benchmarks_sorted
        ]
    )
    misses_diffs.extend(
        [
            misses_diff.get(bm if "amean" not in bm else "amean")
            for bm in benchmarks_sorted
        ]
    )

    # Annotations and dividers
    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Create Plot
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=hits_diffs,
        name="L1I Hits",
        marker_color="mediumseagreen",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=misses_diffs,
        name="L1I Misses",
        marker_color="salmon",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 300],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="v",  # Vertical layout
        x=0.025,  # Left side
        y=0.95,  # Top side
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",  # Optional: semi-transparent white background
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
    font=dict(family="Arial, sans-serif", size=big_size),  # Arial is consistent
)

fig.add_annotation(
    x=17 - 0.25,
    y=300,
    text=f"{hits_diffs[17]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=18 - 0.25,
    y=300,
    text=f"{hits_diffs[18]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=19 - 0.25,
    y=300,
    text=f"{hits_diffs[19]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=20 - 0.25,
    y=300,
    text=f"{hits_diffs[20]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=21 - 0.25,
    y=300,
    text=f"{hits_diffs[21]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
    textangle=-35,
)

fig.add_annotation(
    x=26 + 0.25,
    y=300,
    text=f"{misses_diffs[26]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
)

fig.add_annotation(
    x=28 + 0.25,
    y=300,
    text=f"{misses_diffs[28]:.1f}%",
    showarrow=False,
    font=dict(size=14),
    yshift=10,
)


fig.show()


# Save the figure as an interactive HTML file
fig.write_html("html/l1i_hits_misses_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image(
    "pdf/l1i_hits_misses_plot.pdf", format="pdf", width=1100, height=300, scale=2
)

### L1D, L2C, LLC Misses

In [27]:
levels = ["L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

# For each suite, process the data
for suite in suites:
    # Load MPKI data and drop "gmean" from the indices
    mpki = results[f"{suite}_cp_default_df"]["MPKI"]
    benchmarks = list(mpki.index.drop("gmean", errors="ignore"))

    # Sort the benchmarks except 'amean'
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x], reverse=False)

    # Explicitly move 'amean' to the end
    if "amean" in benchmarks_sorted:
        benchmarks_sorted.remove("amean")
    benchmarks_sorted.append("amean")

    # Label benchmarks, appending the suite prefix
    labeled_benchmarks = [f"{suite.upper()}_{bm}" for bm in benchmarks_sorted]
    all_benchmarks.extend(labeled_benchmarks)

    # For each level, compute misses diffs
    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][f"{level}_TOTAL_MISS"]
        wp_misses = results[f"{suite}_wp_default_df"][f"{level}_TOTAL_MISS"]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")  # Ensure we drop gmean
        misses_diffs_by_level[level].extend(diff[benchmarks_sorted])

    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Plot
fig = go.Figure()

colors = {"L1D": "orange", "L2C": "dodgerblue", "LLC": "mediumorchid"}

for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level} Misses",
            marker_color=colors[level],
        )
    )

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=30, color="black"),
    )

all_benchmarks = [
    # LCF
    "web",
    "media",
    "jbb",
    "wiki",
    # "xap",
    "f-http",
    "speedo",
    "data",
    "kafka",
    "tpcc",
    "cass",
    "f-chirp",
    "vrlatr",
    "tmct",
    "LCF_amean",
    # GAP
    "sssp",
    "pr",
    "prspmv",
    "cc",
    "bc",
    "ccsv",
    "bfs",
    "tc",
    "GAPS_amean",
    # SPEC
    "ex2",
    "x264",
    "sjeng",
    "xz",
    "leela",
    "mcf",
    "SPEC_amean",
]

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 90],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        # x=0.025,
        # y=0.95,
        # put it at the top center outside the plot
        x=0.5,
        y=1.5,
        xanchor="center",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=28),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=10, b=20),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

# Add annotation for a specific benchmark if necessary
fig.add_annotation(
    x=11 + 0.33,
    y=90,
    text=f"{misses_diffs_by_level['LLC'][11]:.1f}%",
    showarrow=False,
    font=dict(size=24),
    yshift=10,
)

fig.show()

# Save output
fig.write_html("html/misses_L1D_L2C_LLC.html", include_plotlyjs="cdn")
fig.write_image(
    "pdf/misses_L1D_L2C_LLC.pdf", format="pdf", width=1100, height=400, scale=2
)

### L1D, L2C, LLC Hits

In [33]:
levels = ["L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]
hits_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    # Get the benchmarks and drop "gmean" from the indices
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    # Sort the benchmarks by MPKI (or any metric you're interested in)
    mpki = results[f"{suite}_cp_default_df"]["MPKI"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x], reverse=False)

    # Explicitly move 'amean' to the end
    if "amean" in benchmarks_sorted:
        benchmarks_sorted.remove("amean")
    benchmarks_sorted.append("amean")

    # Label the benchmarks, adding the suite prefix except for 'amean'
    labeled_benchmarks = [f"{suite.upper()}_{bm}" for bm in benchmarks_sorted]
    all_benchmarks.extend(labeled_benchmarks)

    # For each level, calculate the difference in hits
    for level in levels:
        cp_hits = results[f"{suite}_cp_default_df"][f"{level}_TOTAL_HITS"]
        wp_hits = results[f"{suite}_wp_default_df"][f"{level}_TOTAL_HITS"]
        diff = ((wp_hits - cp_hits) / cp_hits) * 100
        diff = diff.drop("gmean", errors="ignore")  # Drop gmean from the diff
        hits_diffs_by_level[level].extend(diff[benchmarks_sorted])

    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Plotting
fig = go.Figure()

colors = {"L1D": "orange", "L2C": "dodgerblue", "LLC": "mediumorchid"}

# Add the bars for each level
for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=hits_diffs_by_level[level],
            name=f"{level} Hits",
            marker_color=colors[level],
        )
    )

# Add the suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add the suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=30, color="black"),
    )

all_benchmarks = [
    # LCF
    "web",
    "media",
    "jbb",
    "wiki",
    # "xap",
    "f-http",
    "speedo",
    "data",
    "kafka",
    "tpcc",
    "cass",
    "f-chirp",
    "vrlatr",
    "tmct",
    "LCF_amean",
    # GAP
    "sssp",
    "pr",
    "prspmv",
    "cc",
    "bc",
    "ccsv",
    "bfs",
    "tc",
    "GAPS_amean",
    # SPEC
    "ex2",
    "x264",
    "sjeng",
    "xz",
    "leela",
    "mcf",
    "SPEC_amean",
]

# Layout settings for better visibility
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-20, 90],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        x=0.5,
        y=1.5,
        xanchor="center",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=28),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=10, b=20),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

# Annotations for specific values (optional, based on your needs)
fig.add_annotation(
    x=11 + (0.33 / 2.0),
    y=90,
    text=f"{hits_diffs_by_level['L2C'][11]:.1f}%",
    showarrow=False,
    font=dict(size=24),
    yshift=10,
)

fig.add_annotation(
    x=11 + 0.33,
    y=10,
    text=f"{hits_diffs_by_level['LLC'][11]:.1f}%",
    showarrow=False,
    font=dict(size=24),
    yshift=10,
    textangle=-75,
)

fig.add_annotation(
    x=19 - (0.33 / 2.0),
    y=90,
    text=f"{hits_diffs_by_level['L1D'][19]:.1f}%",
    showarrow=False,
    font=dict(size=24),
    yshift=10,
)

fig.show()

# Save the plot
fig.write_html("html/hits_L1D_L2C_LLC.html", include_plotlyjs="cdn")
fig.write_image(
    "pdf/hits_L1D_L2C_LLC.pdf", format="pdf", width=1100, height=400, scale=2
)

### L1I, L1D, L2C, LLC CP

In [ ]:
levels = ["L1I", "L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    # Get the benchmarks and drop "gmean" from the indices
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    # Sort the benchmarks by MPKI (or any metric you're interested in)
    mpki = results[f"{suite}_cp_default_df"]["MPKI"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x], reverse=False)

    # Explicitly move 'amean' to the end
    if "amean" in benchmarks_sorted:
        benchmarks_sorted.remove("amean")
    benchmarks_sorted.append("amean")

    # Label the benchmarks, adding the suite prefix except for 'amean'
    labeled_benchmarks = [f"{suite.upper()}_{bm}" for bm in benchmarks_sorted]
    all_benchmarks.extend(labeled_benchmarks)

    # For each level, calculate the difference in misses
    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][f"{level}_POLLUTION_CP_MISS"]
        wp_misses = results[f"{suite}_wp_default_df"][f"{level}_POLLUTION_CP_MISS"]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")  # Drop gmean from the diff
        misses_diffs_by_level[level].extend(diff[benchmarks_sorted])

    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Plotting
fig = go.Figure()

colors = {"L1I": "green", "L1D": "orange", "L2C": "dodgerblue", "LLC": "mediumorchid"}

# Add the bars for each level
for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level}",
            marker_color=colors[level],
        )
    )

# Add the suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add the suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

# Layout settings for better visibility
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="No-WP Miss Reduction (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-100, 50],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        x=0.025,
        y=0.15,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
    font=dict(family="Arial, sans-serif", size=big_size),  # Arial is consistent
)

fig.show()

# Save the plot
fig.write_html("html/L1I_L1D_L2C_LLC.html", include_plotlyjs="cdn")
fig.write_image(
    "pdf/L1I_L1D_L2C_LLC.pdf", format="pdf", width=1100, height=400, scale=2
)

### ROB at Misspredict

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
rob_full_diff = []  # List for storing differences
cp_rob_at_miss_all = []
wp_rob_at_miss_all = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    # Load metrics
    cp_rob_at_miss = results[f"{suite}_cp_default_df"][
        "Average_ROB_Occupancy_at_Mispredict"
    ]
    wp_rob_at_miss = results[f"{suite}_wp_default_df"][
        "Average_ROB_Occupancy_at_Mispredict"
    ]

    cp_rob_full = results[f"{suite}_cp_default_df"]["ROB_Full_Events"]
    wp_rob_full = results[f"{suite}_wp_default_df"]["ROB_Full_Events"]

    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean from all metrics
    for metric in [cp_rob_at_miss, wp_rob_at_miss, cp_rob_full, wp_rob_full, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Get benchmark list (exclude amean)
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    sorted_benchmarks = sorted(benchmarks, key=lambda x: mpki[x])

    # Collect metric values *in the sorted order*
    cp_vals = [cp_rob_at_miss[bm] for bm in sorted_benchmarks]
    wp_vals = [wp_rob_at_miss[bm] for bm in sorted_benchmarks]
    rob_diff_vals = [
        ((wp_rob_full[bm] - cp_rob_full[bm]) / cp_rob_full[bm]) * 100
        for bm in sorted_benchmarks
    ]

    # Add amean if available
    if "amean" in mpki.index:
        amean_label = f"{suite.upper()}_amean"
        sorted_benchmarks.append(amean_label)
        cp_vals.append(cp_rob_at_miss["amean"])
        wp_vals.append(wp_rob_at_miss["amean"])
        rob_diff_vals.append(
            ((wp_rob_full["amean"] - cp_rob_full["amean"]) / cp_rob_full["amean"]) * 100
        )

    # Label benchmarks
    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in sorted_benchmarks
    ]

    # Extend global lists
    all_benchmarks.extend(labeled_benchmarks)
    cp_rob_at_miss_all.extend(cp_vals)
    wp_rob_at_miss_all.extend(wp_vals)
    rob_full_diff.extend(rob_diff_vals)

    # Labels and dividers
    suite_labels.append(
        (
            x_position + len(sorted_benchmarks) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(sorted_benchmarks)
    suite_boundaries.append(x_position - 0.5)


# Create Plot
fig = go.Figure()

all_benchmarks = [
    # LCF
    "web",
    "media",
    "jbb",
    "wiki",
    # "xap",
    "f-http",
    "speedo",
    "data",
    "kafka",
    "tpcc",
    "cass",
    "f-chirp",
    "vrlatr",
    "tmct",
    "LCF_amean",
    # GAP
    "sssp",
    "pr",
    "prspmv",
    "cc",
    "bc",
    "ccsv",
    "bfs",
    "tc",
    "GAPS_amean",
    # SPEC
    "ex2",
    "x264",
    "sjeng",
    "xz",
    "leela",
    "mcf",
    "SPEC_amean",
]

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=cp_rob_at_miss_all,
        name="No-WP",
        marker_color="mediumseagreen",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=wp_rob_at_miss_all,
        name="ED-WP",
        marker_color="salmon",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=30, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        # title="Average ROB Occupancy at Mispredict",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    barmode="group",
    legend=dict(
        orientation="v",  # Vertical layout
        x=0.025,  # Left side
        y=0.95,  # Top side
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",  # Optional: semi-transparent white background
        bordercolor="black",
        borderwidth=1,
        font=dict(size=28),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
    font=dict(family="Arial, sans-serif", size=30),  # Arial is consistent
)

fig.show()

# Save the figure as an interactive HTML file
fig.write_html("html/rob_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("pdf/rob_plot.pdf", format="pdf", width=1100, height=400, scale=2)

### ROB Full Events

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
rob_pki_cp_all = []
rob_pki_wp_all = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    cp_df = results[f"{suite}_cp_default_df"]
    wp_df = results[f"{suite}_wp_default_df"]

    cp_rob_full = cp_df["ROB_Full_Events"]
    wp_rob_full = wp_df["ROB_Full_Events"]
    mpki = wp_df["MPKI"]

    # Drop gmean from all
    for metric in [cp_rob_full, wp_rob_full, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Sort by MPKI
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    sorted_benchmarks = sorted(benchmarks, key=lambda x: mpki[x])

    # Compute PKI using known 100M instructions
    cp_vals = [cp_rob_full[bm] / 100_000 for bm in sorted_benchmarks]
    wp_vals = [wp_rob_full[bm] / 100_000 for bm in sorted_benchmarks]

    # Append amean if available
    if "amean" in mpki.index:
        amean_label = f"{suite.upper()}_amean"
        sorted_benchmarks.append(amean_label)
        cp_vals.append(cp_rob_full["amean"] / 100_000)
        wp_vals.append(wp_rob_full["amean"] / 100_000)

    # Label benchmarks
    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in sorted_benchmarks
    ]

    all_benchmarks.extend(labeled_benchmarks)
    rob_pki_cp_all.extend(cp_vals)
    rob_pki_wp_all.extend(wp_vals)

    suite_labels.append(
        (
            x_position + len(sorted_benchmarks) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(sorted_benchmarks)
    suite_boundaries.append(x_position - 0.5)

# Plotting same as before (unchanged)
fig = go.Figure()

all_benchmarks = [
    # LCF
    "web",
    "media",
    "jbb",
    "wiki",
    # "xap",
    "f-http",
    "speedo",
    "data",
    "kafka",
    "tpcc",
    "cass",
    "f-chirp",
    "vrlatr",
    "tmct",
    "LCF_amean",
    # GAP
    "sssp",
    "pr",
    "prspmv",
    "cc",
    "bc",
    "ccsv",
    "bfs",
    "tc",
    "GAPS_amean",
    # SPEC
    "ex2",
    "x264",
    "sjeng",
    "xz",
    "leela",
    "mcf",
    "SPEC_amean",
]

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=rob_pki_cp_all,
        name="No-WP",
        marker_color="darkcyan",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=rob_pki_wp_all,
        name="ED-WP",
        marker_color="darkorange",
    )
)

for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=30, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        # title="ROB Full Events per Kilo Instructions (PKI)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 25],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="v",
        x=0.025,
        y=0.95,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=28),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=35, b=20),
    font=dict(family="Arial, sans-serif", size=30),  # Arial is consistent
)

# fig.add_annotation(
#     x=16 - 0.25,
#     y=25,
#     text=f"{rob_pki_cp_all[16]:.1f}",
#     showarrow=False,
#     font=dict(size=18),
#     yshift=10,
#     textangle=-35,
# )

# fig.add_annotation(
#     x=16 + 0.33,
#     y=25,
#     text=f"{rob_pki_wp_all[16]:.1f}",
#     showarrow=False,
#     font=dict(size=18),
#     yshift=10,
#     textangle=-35,
# )

# fig.add_annotation(
#     x=17 - 0.25,
#     y=25,
#     text=f"{rob_pki_cp_all[17]:.1f}",
#     showarrow=False,
#     font=dict(size=18),
#     yshift=10,
#     textangle=-35,
# )

# fig.add_annotation(
#     x=17 + 0.33,
#     y=25,
#     text=f"{rob_pki_wp_all[17]:.1f}",
#     showarrow=False,
#     font=dict(size=18),
#     yshift=10,
#     textangle=-35,
# )

fig.show()
fig.write_html("html/rob_pki.html", include_plotlyjs="cdn")
fig.write_image("pdf/rob_pki.pdf", format="pdf", width=1100, height=400, scale=2)

### Speedup for all benchmarks

In [ ]:
suites = ["lcf", "gaps", "spec"]
wp_label = "ED-WP"
wpa_label = "TI-WP"
colors = {"ED-WP": "steelblue", "TI-WP": "indianred"}

all_benchmarks = []
wp_speedups = []
wpa_speedups = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    cp = results[f"{suite}_cp_default_df"]["IPC"]
    wp = results[f"{suite}_wp_default_df"]["IPC"]
    wpa = results[f"{suite}_wpa_default_df"]["IPC"]
    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop amean early
    cp_no_gmean = cp.drop("amean", errors="ignore")
    wp_no_gmean = wp.drop("amean", errors="ignore")
    wpa_no_gmean = wpa.drop("amean", errors="ignore")
    mpki_no_gmean = mpki.drop("amean", errors="ignore")

    # Sort benchmarks by MPKI
    sorted_benchmarks = mpki_no_gmean.sort_values().index.tolist()

    # Apply label (prefix gmean as well)
    benchmarks_labeled = [f"{suite.upper()}_{bm}" for bm in sorted_benchmarks]
    all_benchmarks.extend(benchmarks_labeled)

    # Extend data in the sorted order
    wp_speedups.extend(
        ((wp.reindex(sorted_benchmarks) / cp.reindex(sorted_benchmarks)) - 1) * 100
    )
    wpa_speedups.extend(
        ((wpa.reindex(sorted_benchmarks) / cp.reindex(sorted_benchmarks)) - 1) * 100
    )

    # Save annotation and boundaries
    suite_labels.append(
        (
            x_position + len(sorted_benchmarks) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(sorted_benchmarks)
    suite_boundaries.append(x_position - 0.5)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=all_benchmarks, y=wpa_speedups, name=wpa_label, marker_color=colors["TI-WP"]
    )
)

fig.add_trace(
    go.Bar(x=all_benchmarks, y=wp_speedups, name=wp_label, marker_color=colors["ED-WP"])
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite annotations above
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    # title="WP and WPA Speedup vs CP (Default)",
    xaxis=dict(
        # title=dict(
        #     text="Benchmark",
        #     standoff=50,  # Add vertical space between axis and title
        # ),
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,  # Ensure horizontal grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    yaxis=dict(
        title="Speedup (%)",
        range=[-5, 30],  # max(max(wp_speedups), max(wpa_speedups)) * 1.1
        showgrid=True,  # Ensure vertical grid lines are shown
        gridcolor="lightgray",  # Optional: color of the grid lines
        gridwidth=0.5,  # Optional: width of the grid lines
        zeroline=True,  # Add a zero line
        zerolinecolor="black",  # Color of the zero line
        zerolinewidth=2,  # Width of the zero line
        griddash="dot",
    ),
    barmode="group",
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
    font=dict(family="Arial, sans-serif", size=big_size),  # Arial is consistent
)

index = all_benchmarks.index("GAPS_cc")
index_value = wp_speedups[index]
fig.add_annotation(
    x=index + 0.25,
    y=30,
    text=f"{index_value:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
    textangle=-35,
)

index = all_benchmarks.index("GAPS_bc")
index_value = wp_speedups[index]
fig.add_annotation(
    x=index + 0.25,
    y=30,
    text=f"{index_value:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
    textangle=-35,
)

index = all_benchmarks.index("GAPS_ccsv")
index_value = wp_speedups[index]
fig.add_annotation(
    x=index + 0.25,
    y=30,
    text=f"{index_value:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
    textangle=-35,
)

index = all_benchmarks.index("GAPS_bfs")
index_value = wp_speedups[index]
fig.add_annotation(
    x=index + 0.25,
    y=30,
    text=f"{index_value:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
    textangle=-35,
)

fig.add_shape(
    type="line",
    x0=0,
    x1=1,
    y0=30,  # Match the y-axis max
    y1=30,
    xref="paper",  # Use the whole width of the plot
    yref="y",
    line=dict(color="black", width=2, dash="solid"),  # You can use 'dot', 'dash', etc.
    layer="above",
)

fig.show()

# Save the figure as an interactive HTML file
fig.write_html("html/speedup_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("pdf/speedup_plot.pdf", format="pdf", width=1100, height=400, scale=2)

### L1I CP v WP

In [ ]:
from plotly import graph_objects as go

suites = ["lcf"]

pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

colorblind_colors = [
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#F0E442",
    "#0072B2",
    "#D55E00",
    "#CC79A7",
    "#999999",
    "#000000",
]

prefetchers = {
    "l1i-barsa": {"label": "BARÇA", "color": colorblind_colors[0]},
    "l1i-bip": {"label": "BIP", "color": colorblind_colors[1]},
    "l1i-djolt": {"label": "Djolt", "color": colorblind_colors[2]},
    "l1i-epi": {"label": "EIP", "color": colorblind_colors[3]},
    "l1i-fnlmma": {"label": "FNL+MMA", "color": colorblind_colors[4]},
    "l1i-mana": {"label": "Mana", "color": colorblind_colors[5]},
    "l1i-next_line": {"label": "Next Line", "color": colorblind_colors[6]},
    "l1i-pips": {"label": "PIPS", "color": colorblind_colors[7]},
    "l1i-tap": {"label": "TAP", "color": colorblind_colors[8]},
}

# Build MPKI-based order from a reference group (e.g., CP)
mpki_values = results["lcf_wp_default_df"].get("MPKI", {})
sorted_prefetchers = sorted(
    [pf for pf in prefetchers if pf in results["lcf_wp_default_df"]],
    key=lambda pf: mpki_values.get(pf, float("inf")),
)

# Append 'amean' if it's present
if "amean" in results["lcf_wp_default_df"].get("IPC", {}):
    sorted_prefetchers.append("amean")

suite_labels = ["Correct Path", "Wrong Path"]
speedups_by_prefetcher = {
    pf: {"cp": [], "wp": []} for pf in prefetchers
}  # Ensure CP, WP are initialized

for suite in suites:
    # Loop through CP, WP
    for group in ["cp", "wp"]:
        default_baseline = (
            results.get(f"{suite}_{group}_default_df", {})
            .get("IPC", {})
            .get("gmean", None)
        )
        if default_baseline is None:
            print(
                f"Warning: {suite}_{group}_default_df data is missing or incorrectly formatted."
            )
            continue

        for pf in prefetchers:
            p_pf = (
                results.get(f"{suite}_{group}_{pf}_df", {})
                .get("IPC", {})
                .get("gmean", None)
            )
            if p_pf is None:
                print(
                    f"Warning: {suite}_{group}_{pf}_df data is missing or incorrectly formatted."
                )
                continue

            speedup = ((p_pf / default_baseline) - 1) * 100
            speedups_by_prefetcher[pf][group].append(speedup)

# --- Create the first figure for Correct Path ---
fig_cp = go.Figure()

# Plot for Correct Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_cp.add_trace(
        go.Bar(
            x=[props["label"]],  # Prefetcher labels as x-axis
            y=[speedups_by_prefetcher[pf]["cp"][0]],  # Correct Path speedup
            name=props["label"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            marker_color=props["color"],
        )
    )

# Update layout for Correct Path figure
fig_cp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-2, 7],  # Adjust range for the first row if needed
    ),
    barmode="group",  # Group bars within each row
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.15,
        xanchor="center",
        x=0.5,
        font=dict(size=small_size),
    ),
    showlegend=False,
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

# Add annotations for Correct Path
fig_cp.add_annotation(
    x="EIP",
    y=7,
    text=f"{speedups_by_prefetcher['l1i-epi']['cp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.add_annotation(
    x="BIP",
    y=0,
    text=f"{speedups_by_prefetcher['l1i-bip']['cp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the first figure (Correct Path)
fig_cp.show()

# Save the first figure as an interactive HTML file
fig_cp.write_html("html/l1i_prefetchers_cp_plot.html", include_plotlyjs="cdn")

# Save the first figure as a static PDF
fig_cp.write_image(
    "pdf/l1i_prefetchers_cp_plot.pdf", format="pdf", width=1200, height=400, scale=2
)


# --- Create the second figure for Wrong Path ---
fig_wp = go.Figure()

# Plot for Wrong Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_wp.add_trace(
        go.Bar(
            x=[props["label"]],  # Prefetcher labels as x-axis
            y=[speedups_by_prefetcher[pf]["wp"][0]],  # Wrong Path speedup
            name=props["label"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            marker_color=props["color"],
        )
    )

# Update layout for Wrong Path figure
fig_wp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-2, 7],  # Adjust range for the second row if needed
    ),
    barmode="group",  # Group bars within each row
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.15,
        xanchor="center",
        x=0.5,
        font=dict(size=small_size),
    ),
    showlegend=False,
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

# Add annotations for Wrong Path
fig_wp.add_annotation(
    x="BIP",
    y=0,
    text=f"{speedups_by_prefetcher['l1i-bip']['wp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.add_annotation(
    x="TAP",
    y=0,
    text=f"{speedups_by_prefetcher['l1i-tap']['wp'][0]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the second figure (Wrong Path)
fig_wp.show()

# Save the second figure as an interactive HTML file
fig_wp.write_html("html/l1i_prefetchers_wp_plot.html", include_plotlyjs="cdn")

# Save the second figure as a static PDF
fig_wp.write_image(
    "pdf/l1i_prefetchers_wp_plot.pdf", format="pdf", width=1200, height=400, scale=2
)

### L1D CP vs WP

In [ ]:
from plotly import graph_objects as go

# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]
colorblind_colors = ["#99C945", "#CC61B0", "#24796C", "#DAA51B"]

# Prefetchers dictionary
prefetchers = {
    "l1d-berti": {"label": "Berti", "color": colorblind_colors[0]},
    "l1d-ip_stride": {"label": "IP Stride", "color": colorblind_colors[1]},
    "l1d-ipcp": {"label": "IPCP", "color": colorblind_colors[2]},
    "l1d-next_line": {"label": "Next Line", "color": colorblind_colors[3]},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]
suite_labels[1] = "GAP"  # Special case for "gaps"

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# --- Create the first figure for Correct Path ---
fig_cp = go.Figure()

# Plot for Correct Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_cp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        )
    )

    fig_cp.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

    fig_cp.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )


# Update layout for Correct Path figure
fig_cp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-7, 7],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

fig_cp.add_annotation(
    x=1 - 0.29,
    y=7,
    text=f"{speedups_by_prefetcher['l1d-berti']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.add_annotation(
    x=1 + 0.30,
    y=0,
    text=f"{speedups_by_prefetcher['l1d-next_line']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the first figure (Correct Path)
fig_cp.show()

# Save the first figure as an interactive HTML file
fig_cp.write_html("html/l1d_prefetchers_cp_plot.html", include_plotlyjs="cdn")

# Save the first figure as a static PDF
fig_cp.write_image(
    "pdf/l1d_prefetchers_cp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)


# --- Create the second figure for Wrong Path ---
fig_wp = go.Figure()

# Plot for Wrong Path
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]

    fig_wp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        )
    )

    fig_wp.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

    fig_wp.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )


# Update layout for Wrong Path figure
fig_wp.update_layout(
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-7, 7],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

fig_wp.add_annotation(
    x=1 - 0.29,
    y=6.3,
    text=f"{speedups_by_prefetcher['l1d-berti']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.add_annotation(
    x=1 + 0.30,
    y=0,
    text=f"{speedups_by_prefetcher['l1d-next_line']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

# Show the second figure (Wrong Path)
fig_wp.show()

# Save the second figure as an interactive HTML file
fig_wp.write_html("html/l1d_prefetchers_wp_plot.html", include_plotlyjs="cdn")

# Save the second figure as a static PDF
fig_wp.write_image(
    "pdf/l1d_prefetchers_wp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

### L2C CP v WP

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

colorblind_13_colors = [
    "#E69F00",  # orange
    "#56B4E9",  # sky blue
    "#009E73",  # bluish green
    "#F0E442",  # yellow
    "#0072B2",  # blue
    "#D55E00",  # vermillion
    "#CC79A7",  # reddish purple
    "#999999",  # gray
    "#000000",  # black
    "#009E73",  # green
    "#F0E442",  # yellow
    "#D55E00",  # red
    "#56B4E9",  # light blue
]

# Prefetchers dictionary
prefetchers = {
    "l2c-bingo": {"label": "Bingo", "color": colorblind_13_colors[0]},
    "l2c-ip_stride": {"label": "IP Stride", "color": colorblind_13_colors[3]},
    "l2c-mlop": {"label": "MLOP", "color": colorblind_13_colors[5]},
    "l2c-next_line": {"label": "Next Line", "color": colorblind_13_colors[6]},
    "l2c-sms": {"label": "SMS", "color": colorblind_13_colors[9]},
    "l2c-spp": {"label": "SPP", "color": colorblind_13_colors[10]},
    "l2c-streamer": {"label": "Streamer", "color": colorblind_13_colors[12]},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]
suite_labels[1] = "GAP"  # Special case for "gaps"

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# Plot for CP speedup (first figure)
fig_cp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_cp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        )
    )

# Add vertical lines to CP plot
for i in range(1, len(suite_labels)):
    fig_cp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_cp.update_layout(
    # title="CP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-8, 8],
        dtick=2,
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

fig_cp.add_annotation(
    x=1 - 0.34,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-bingo']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.add_annotation(
    x=1,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-next_line']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_cp.show()

# Save the figure as an interactive HTML file for CP
fig_cp.write_html("html/l2c_prefetchers_cp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF for CP
fig_cp.write_image(
    "pdf/l2c_prefetchers_cp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

# Plot for WP speedup (second figure)
fig_wp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_wp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        )
    )

# Add vertical lines to WP plot
for i in range(1, len(suite_labels)):
    fig_wp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_wp.update_layout(
    # title="WP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-8, 8],
        dtick=2,
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

fig_wp.add_annotation(
    x=1 - 0.34,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-bingo']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.add_annotation(
    x=1,
    y=0,
    text=f"{speedups_by_prefetcher['l2c-next_line']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig_wp.show()

# Save the figure as an interactive HTML file for WP
fig_wp.write_html("html/l2c_prefetchers_wp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF for WP
fig_wp.write_image(
    "pdf/l2c_prefetchers_wp_plot.pdf", format="pdf", width=1100, height=400, scale=2
)

### LLC Repl CP v WP

In [ ]:
import plotly.graph_objects as go

# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

# Prefetchers dictionary
prefetchers = {
    "llc-ship": {"label": "SHIP", "color": "blue"},
    "llc-srrip": {"label": "SRRIP", "color": "green"},
    "llc-drrip": {"label": "DRRIP", "color": "orange"},
    "llc-mockingjay": {"label": "Mockingjay", "color": "purple"},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]
suite_labels[1] = "GAP"  # Special case for "gaps"

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# CP Speedup plot
fig_cp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_cp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        )
    )

# Add vertical lines to CP plot
for i in range(1, len(suite_labels)):
    fig_cp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_cp.update_layout(
    # title="CP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - No-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 10],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

# Show the plot for CP
fig_cp.show()

# Save the figure as an interactive HTML file
fig_cp.write_html("html/llc_cp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig_cp.write_image("pdf/llc_cp_plot.pdf", format="pdf", width=1100, height=400, scale=2)

# WP Speedup plot
fig_wp = go.Figure()

for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig_wp.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        )
    )

# Add vertical lines to WP plot
for i in range(1, len(suite_labels)):
    fig_wp.add_vline(
        x=i - 0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig_wp.update_layout(
    # title="WP Speedup",
    xaxis=dict(
        tickangle=0,
        tickfont=dict(size=small_size),
    ),
    yaxis=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 10],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
    font=dict(family="Arial, sans-serif", size=small_size),  # Arial is consistent
)

# Show the plot for WP
fig_wp.show()

# Save the figure as an interactive HTML file
fig_wp.write_html("html/llc_wp_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig_wp.write_image("pdf/llc_wp_plot.pdf", format="pdf", width=1100, height=400, scale=2)

### L1I footprint

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
percent_increase = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    # Load data
    if suite != "lcf":
        l1i_cp = results[f"{suite}_cp_default_df"]["instr_foot_print"]
        l1i_wp = results[f"{suite}_wp_default_df"]["instr_foot_print"]
    else:
        l1i_cp = results[f"{suite}_cp_default_df"]["instr_foot_print"]
        l1i_wp = results[f"{suite}_wp_default_df"]["instr_foot_print"]

    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean
    for metric in [l1i_cp, l1i_wp, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Get benchmark list excluding amean
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x])

    # Add suite-prefixed amean at the end if available
    if "amean" in mpki.index:
        benchmarks_sorted.append(f"{suite.upper()}_amean")

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in benchmarks_sorted
    ]

    all_benchmarks.extend(labeled_benchmarks)

    for bm in benchmarks_sorted:
        base_bm = "amean" if "amean" in bm else bm
        cp_val = l1i_cp.get(base_bm, 0)
        wp_val = l1i_wp.get(base_bm, 0)
        pct = ((wp_val - cp_val) / cp_val * 100) if cp_val != 0 else 0
        percent_increase.append(pct)

    # Annotations and dividers
    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Create Plot
fig = go.Figure()


all_benchmarks = [
    # LCF
    "web",
    "media",
    "jbb",
    "wiki",
    # "xap",
    "f-http",
    "speedo",
    "data",
    "kafka",
    "tpcc",
    "cass",
    "f-chirp",
    "vrlatr",
    "tmct",
    "LCF_amean",
    # GAP
    "sssp",
    "pr",
    "prspmv",
    "cc",
    "bc",
    "ccsv",
    "bfs",
    "tc",
    "GAPS_amean",
    # SPEC
    "ex2",
    "x264",
    "sjeng",
    "xz",
    "leela",
    "mcf",
    "SPEC_amean",
]

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=percent_increase,
        name="L1I ED-WP vs No-WP",
        marker_color="darkorange",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=small_size, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        # title="% Increase",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 180],
        dtick=60,
    ),
    barmode="group",
    legend=dict(
        orientation="v",
        x=0.025,
        y=0.95,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
    font=dict(family="Arial, sans-serif", size=30),  # Arial is consistent
)

fig.show()

# Save
fig.write_html("html/footprint.html", include_plotlyjs="cdn")
fig.write_image("pdf/footprint.pdf", format="pdf", width=1100, height=500, scale=3)

# WIP

### L1I & L1D footprint

In [ ]:
suites = ["lcf", "gaps", "spec"]
all_benchmarks = []
all_l1i_cp = []
all_l1i_wp = []
all_l1d_cp = []
all_l1d_wp = []
misses_diffs = []
suite_boundaries = []
suite_labels = []

x_position = 0

for suite in suites:
    # Load data
    if suite != "lcf":
        l1i_cp = results[f"{suite}_cp_default_df"]["instr_foot_print"] * 64 / 1024
        l1i_wp = results[f"{suite}_wp_default_df"]["instr_foot_print"] * 64 / 1024
        l1d_cp = results[f"{suite}_cp_default_df"]["data_foot_print"] * 64 / 1024
        l1d_wp = results[f"{suite}_wp_default_df"]["data_foot_print"] * 64 / 1024
    else:
        l1i_cp = results[f"{suite}_cp_default_df"]["instr_foot_print"] * 64 / 1024**2
        l1i_wp = results[f"{suite}_wp_default_df"]["instr_foot_print"] * 64 / 1024**2
        l1d_cp = results[f"{suite}_cp_default_df"]["data_foot_print"] * 64 / 1024**2
        l1d_wp = results[f"{suite}_wp_default_df"]["data_foot_print"] * 64 / 1024**2

    mpki = results[f"{suite}_wp_default_df"]["MPKI"]

    # Drop gmean
    for metric in [l1i_cp, l1i_wp, l1d_cp, l1d_wp, mpki]:
        metric.drop("gmean", errors="ignore", inplace=True)

    # Get benchmark list excluding amean
    benchmarks = [bm for bm in mpki.index if bm != "amean"]
    benchmarks_sorted = sorted(benchmarks, key=lambda x: mpki[x])

    # Add suite-prefixed amean at the end if available
    if "amean" in mpki.index:
        benchmarks_sorted.append(f"{suite.upper()}_amean")

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm == "amean" else bm for bm in benchmarks_sorted
    ]

    all_benchmarks.extend(labeled_benchmarks)

    all_l1i_cp.extend(
        [l1i_cp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )
    all_l1i_wp.extend(
        [l1i_wp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )
    all_l1d_cp.extend(
        [l1d_cp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )
    all_l1d_wp.extend(
        [l1d_wp.get(bm if "amean" not in bm else "amean") for bm in benchmarks_sorted]
    )

    # all_l1i_cp.extend(l1i_cp)
    # all_l1i_wp.extend(l1i_wp)
    # all_l1d_cp.extend(l1d_cp)
    # all_l1d_wp.extend(l1d_wp)

    # Annotations and dividers
    suite_labels.append(
        (
            x_position + len(benchmarks_sorted) / 2,
            suite.upper() if suite != "gaps" else "GAP",
        )
    )
    x_position += len(benchmarks_sorted)
    suite_boundaries.append(x_position - 0.5)

# Create Plot
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1i_cp,
        name="L1I No-WP",
        marker_color="mediumseagreen",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1i_wp,
        name="L1I ED-WP",
        marker_color="salmon",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1d_cp,
        name="L1D No-WP",
        marker_color="lightblue",
    )
)

fig.add_trace(
    go.Bar(
        x=all_benchmarks,
        y=all_l1d_wp,
        name="L1D ED-WP",
        marker_color="lightcoral",
    )
)

# Add vertical lines to separate suites
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Add suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Increase (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[0, 300],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="v",  # Vertical layout
        x=0.025,  # Left side
        y=0.95,  # Top side
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",  # Optional: semi-transparent white background
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()


# Save the figure as an interactive HTML file
fig.write_html("footprint.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image("footprint.pdf", format="pdf", width=1100, height=400, scale=2)

### AMAT L1I, L2C, LLC, and ITLB

In [ ]:
levels = ["L1I", "L2C", "LLC", "ITLB"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm in ["amean", "gmean"] else bm for bm in benchmarks
    ]
    all_benchmarks.extend(labeled_benchmarks)

    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][
            f"{level}_AVERAGE_CP_INSTR_MISS_LATENCY"
        ]
        wp_misses = results[f"{suite}_wp_default_df"][
            f"{level}_AVERAGE_CP_INSTR_MISS_LATENCY"
        ]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")
        misses_diffs_by_level[level].extend(diff[benchmarks])

    suite_labels.append((x_position + len(benchmarks) / 2, suite.upper()))
    x_position += len(benchmarks)
    suite_boundaries.append(x_position - 0.5)

# Plot
fig = go.Figure()

colors = {
    "L1I": "green",
    "L1D": "orange",
    "L2C": "dodgerblue",
    "LLC": "mediumorchid",
    "ITLB": "red",
}

for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level}",
            marker_color=colors[level],
        )
    )

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="CP Miss Reduction (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-100, 50],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        x=0.025,
        y=0.15,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save output
fig.write_html("AMAT_L1I_L2C_LLC_ITLB.html", include_plotlyjs="cdn")
fig.write_image(
    "AMAT_L1I_L2C_LLC_ITLB.pdf", format="pdf", width=1100, height=400, scale=2
)

In [ ]:
levels = ["L1D", "L2C", "LLC", "DTLB"]
suites = ["lcf", "gaps", "spec"]
misses_diffs_by_level = {level: [] for level in levels}
all_benchmarks = []
suite_labels = []
suite_boundaries = []

x_position = 0

for suite in suites:
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    labeled_benchmarks = [
        f"{suite.upper()}_{bm}" if bm in ["amean", "gmean"] else bm for bm in benchmarks
    ]
    all_benchmarks.extend(labeled_benchmarks)

    for level in levels:
        cp_misses = results[f"{suite}_cp_default_df"][
            f"{level}_AVERAGE_CP_DATA_MISS_LATENCY"
        ]
        wp_misses = results[f"{suite}_wp_default_df"][
            f"{level}_AVERAGE_CP_DATA_MISS_LATENCY"
        ]
        diff = ((wp_misses - cp_misses) / cp_misses) * 100
        diff = diff.drop("gmean", errors="ignore")
        misses_diffs_by_level[level].extend(diff[benchmarks])

    suite_labels.append((x_position + len(benchmarks) / 2, suite.upper()))
    x_position += len(benchmarks)
    suite_boundaries.append(x_position - 0.5)

# Plot
fig = go.Figure()

colors = {
    "L1D": "green",
    "L1D": "orange",
    "L2C": "dodgerblue",
    "LLC": "mediumorchid",
    "DTLB": "red",
}

for level in levels:
    fig.add_trace(
        go.Bar(
            x=all_benchmarks,
            y=misses_diffs_by_level[level],
            name=f"{level}",
            marker_color=colors[level],
        )
    )

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(all_benchmarks))),
        ticktext=[bm.split("_", 1)[-1] for bm in all_benchmarks],
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="CP Miss Reduction (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-100, 50],  # Adjusted range for better visibility
    ),
    barmode="group",
    legend=dict(
        orientation="h",
        x=0.025,
        y=0.15,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save output
fig.write_html("AMAT_L1D_L2C_LLC_DTLB.html", include_plotlyjs="cdn")
fig.write_image(
    "AMAT_L1D_L2C_LLC_DTLB.pdf", format="pdf", width=1100, height=400, scale=2
)

# TO DROP

### Cycle breakdown

In [ ]:
# total cycles
lcf_total_cycles = results["lcf_wp_default_df"]["total_cycles"]
gaps_total_cycles = results["gaps_wp_default_df"]["total_cycles"]
spec_total_cycles = results["spec_wp_default_df"]["total_cycles"]

# cp_cycles
lcf_cp_cycles = results["lcf_wp_default_df"]["Execute_Only_CP_Cycles"]
gaps_cp_cycles = results["gaps_wp_default_df"]["Execute_Only_CP_Cycles"]
spec_cp_cycles = results["spec_wp_default_df"]["Execute_Only_CP_Cycles"]

# wp_cycles
lcf_wp_cycles = results["lcf_wp_default_df"]["Execute_Only_WP_Cycles"]
gaps_wp_cycles = results["gaps_wp_default_df"]["Execute_Only_WP_Cycles"]
spec_wp_cycles = results["spec_wp_default_df"]["Execute_Only_WP_Cycles"]

# cp_wp_cycles
lcf_cp_wp_cycles = results["lcf_wp_default_df"]["Execute_CP_WP_Cycles"]
gaps_cp_wp_cycles = results["gaps_wp_default_df"]["Execute_CP_WP_Cycles"]
spec_cp_wp_cycles = results["spec_wp_default_df"]["Execute_CP_WP_Cycles"]

# rob empty
lcf_rob_empty_cycles = results["lcf_wp_default_df"]["Execute_ROB_Empty_Cycles"]
gaps_rob_empty_cycles = results["gaps_wp_default_df"]["Execute_ROB_Empty_Cycles"]
spec_rob_empty_cycles = results["spec_wp_default_df"]["Execute_ROB_Empty_Cycles"]

# Execute_ROB_Empty_Repair_Cycles
lcf_rob_empty_repair_cycles = results["lcf_wp_default_df"][
    "Execute_ROB_Empty_Repair_Cycles"
]
gaps_rob_empty_repair_cycles = results["gaps_wp_default_df"][
    "Execute_ROB_Empty_Repair_Cycles"
]
spec_rob_empty_repair_cycles = results["spec_wp_default_df"][
    "Execute_ROB_Empty_Repair_Cycles"
]

# Execute_ROB_Empty_Fetch_Stalled_No_WP_Cycles
lcf_rob_empty_fetch_stalled_cycles = results["lcf_wp_default_df"][
    "Execute_ROB_Empty_Fetch_Stalled_No_WP_Cycles"
]
gaps_rob_empty_fetch_stalled_cycles = results["gaps_wp_default_df"][
    "Execute_ROB_Empty_Fetch_Stalled_No_WP_Cycles"
]
spec_rob_empty_fetch_stalled_cycles = results["spec_wp_default_df"][
    "Execute_ROB_Empty_Fetch_Stalled_No_WP_Cycles"
]

# Execute_ROB_Not_Ready_Not_Full_New_Added_Cycles + Execute_ROB_Not_Ready_Not_Full_No_New_Added_Cycles
# + Execute_ROB_Not_Ready_Full_New_Added_Cycles +Execute_ROB_Not_Ready_Full_No_New_Added_Cycles
lcf_rob_not_ready_cycles = (
    results["lcf_wp_default_df"]["Execute_ROB_Not_Ready_Not_Full_New_Added_Cycles"]
    + results["lcf_wp_default_df"]["Execute_ROB_Not_Ready_Not_Full_No_New_Added_Cycles"]
    + results["lcf_wp_default_df"]["Execute_ROB_Not_Ready_Full_New_Added_Cycles"]
    + results["lcf_wp_default_df"]["Execute_ROB_Not_Ready_Full_No_New_Added_Cycles"]
)
gaps_rob_not_ready_cycles = (
    results["gaps_wp_default_df"]["Execute_ROB_Not_Ready_Not_Full_New_Added_Cycles"]
    + results["gaps_wp_default_df"][
        "Execute_ROB_Not_Ready_Not_Full_No_New_Added_Cycles"
    ]
    + results["gaps_wp_default_df"]["Execute_ROB_Not_Ready_Full_New_Added_Cycles"]
    + results["gaps_wp_default_df"]["Execute_ROB_Not_Ready_Full_No_New_Added_Cycles"]
)
spec_rob_not_ready_cycles = (
    results["spec_wp_default_df"]["Execute_ROB_Not_Ready_Not_Full_New_Added_Cycles"]
    + results["spec_wp_default_df"][
        "Execute_ROB_Not_Ready_Not_Full_No_New_Added_Cycles"
    ]
    + results["spec_wp_default_df"]["Execute_ROB_Not_Ready_Full_New_Added_Cycles"]
    + results["spec_wp_default_df"]["Execute_ROB_Not_Ready_Full_No_New_Added_Cycles"]
)


# Divide all the data by "total_cycles"
lcf_cp_cycles = lcf_cp_cycles / lcf_total_cycles
gaps_cp_cycles = gaps_cp_cycles / gaps_total_cycles
spec_cp_cycles = spec_cp_cycles / spec_total_cycles
lcf_wp_cycles = lcf_wp_cycles / lcf_total_cycles
gaps_wp_cycles = gaps_wp_cycles / gaps_total_cycles
spec_wp_cycles = spec_wp_cycles / spec_total_cycles
lcf_cp_wp_cycles = lcf_cp_wp_cycles / lcf_total_cycles
gaps_cp_wp_cycles = gaps_cp_wp_cycles / gaps_total_cycles
spec_cp_wp_cycles = spec_cp_wp_cycles / spec_total_cycles
lcf_rob_empty_cycles = lcf_rob_empty_cycles / lcf_total_cycles
gaps_rob_empty_cycles = gaps_rob_empty_cycles / gaps_total_cycles
spec_rob_empty_cycles = spec_rob_empty_cycles / spec_total_cycles
lcf_rob_empty_repair_cycles = lcf_rob_empty_repair_cycles / lcf_total_cycles
gaps_rob_empty_repair_cycles = gaps_rob_empty_repair_cycles / gaps_total_cycles
spec_rob_empty_repair_cycles = spec_rob_empty_repair_cycles / spec_total_cycles
lcf_rob_empty_fetch_stalled_cycles = (
    lcf_rob_empty_fetch_stalled_cycles / lcf_total_cycles
)
gaps_rob_empty_fetch_stalled_cycles = (
    gaps_rob_empty_fetch_stalled_cycles / gaps_total_cycles
)
spec_rob_empty_fetch_stalled_cycles = (
    spec_rob_empty_fetch_stalled_cycles / spec_total_cycles
)
lcf_rob_not_ready_cycles = lcf_rob_not_ready_cycles / lcf_total_cycles
gaps_rob_not_ready_cycles = gaps_rob_not_ready_cycles / gaps_total_cycles
spec_rob_not_ready_cycles = spec_rob_not_ready_cycles / spec_total_cycles

In [ ]:
import plotly.graph_objects as go
import pandas as pd


# Helper: sort benchmarks by MPKI within suite (exclude "gmean" and "amean")
def get_sorted_benchmarks(suite):
    df = results[f"{suite}_wp_default_df"]
    mpki_sorted = df.drop(index=["gmean", "amean"], errors="ignore").sort_values("MPKI")
    return [f"{suite}_{name}" for name in mpki_sorted.index]


# Get sorted benchmarks per suite
lcf_benchmarks = get_sorted_benchmarks("lcf")
gaps_benchmarks = get_sorted_benchmarks("gaps")
spec_benchmarks = get_sorted_benchmarks("spec")

# Combine benchmark names and append single 'amean' (for total)
benchmarks = lcf_benchmarks + gaps_benchmarks + spec_benchmarks + ["amean"]

# Display names
display_names = [
    name.split("_", 1)[1] if name != "amean" else "amean" for name in benchmarks
]


# Helper: extract values from per-suite series for each benchmark name
def filter_data(lcf_series, gaps_series, spec_series):
    full_series = {
        "lcf": lcf_series,
        "gaps": gaps_series,
        "spec": spec_series,
    }
    values = []
    for name in benchmarks:
        if name == "amean":
            all_values = pd.concat([lcf_series, gaps_series, spec_series])
            values.append(all_values.mean())
        else:
            suite, short = name.split("_", 1)
            values.append(full_series[suite][short])
    return values


# Construct cycle components
cycle_data = {
    "CP": filter_data(lcf_cp_cycles, gaps_cp_cycles, spec_cp_cycles),
    "WP": filter_data(lcf_wp_cycles, gaps_wp_cycles, spec_wp_cycles),
    "CP+WP": filter_data(lcf_cp_wp_cycles, gaps_cp_wp_cycles, spec_cp_wp_cycles),
    "ROB Empty": [
        a + b
        for a, b in zip(
            filter_data(
                lcf_rob_empty_cycles, gaps_rob_empty_cycles, spec_rob_empty_cycles
            ),
            filter_data(
                lcf_rob_empty_repair_cycles,
                gaps_rob_empty_repair_cycles,
                spec_rob_empty_repair_cycles,
            ),
        )
    ],
    "ROB Empty (No WP)": filter_data(
        lcf_rob_empty_fetch_stalled_cycles,
        gaps_rob_empty_fetch_stalled_cycles,
        spec_rob_empty_fetch_stalled_cycles,
    ),
    "ROB Not Ready": filter_data(
        lcf_rob_not_ready_cycles, gaps_rob_not_ready_cycles, spec_rob_not_ready_cycles
    ),
}

# Plotting
fig = go.Figure()

for label, values in cycle_data.items():
    fig.add_trace(
        go.Bar(
            name=label,
            x=display_names,
            y=values,
            hovertemplate=f"%{{y:.2%}}<extra>{label}</extra>",
        )
    )

fig.update_layout(
    barmode="stack",
    yaxis_title="Proportion of Total Cycles",
    yaxis_tickformat=".0%",
    xaxis_tickangle=-45,
    legend_title="Cycle Type",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5,
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    font=dict(family="Arial, sans-serif", size=big_size),
)

# Suite boundary lines (between sorted sections)
boundary_indices = [
    len(lcf_benchmarks) - 0.5,
    len(lcf_benchmarks) + len(gaps_benchmarks) - 0.5,
    len(lcf_benchmarks) + len(gaps_benchmarks) + len(spec_benchmarks) - 0.5,
]

for x in boundary_indices:
    fig.add_vline(
        x=x,
        line_width=2,
        line_dash="dash",
        line_color="gray",
    )

fig.show()

# Save the plot
fig.write_html("html/stacked_cycles.html", include_plotlyjs="cdn")
fig.write_image("pdf/stacked_cycles.pdf", format="pdf", width=1100, height=400, scale=2)

In [ ]:
# Define patterns and colors
pattern_shapes = ["", "/", "\\", "x", "-", "|", "+", "."]

# Prefetchers dictionary
prefetchers = {
    "llc-ip_stride": {"label": "IP Stride", "color": "#E69F00"},
    "llc-next_line": {"label": "Next Line", "color": "#56B4E9"},
}

# Suites to be used
suites = ["lcf", "gaps", "spec"]
suite_labels = [s.upper() for s in suites]

# Initialize dictionary to hold speedup data
speedups_by_prefetcher = {pf: {"cp": [], "wp": []} for pf in prefetchers}

# Calculate speedups for both CP and WP
for suite in suites:
    cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
    wp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
    for pf in prefetchers:
        cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
        wp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
        speedup_cp = ((cp_pf / cp_default) - 1) * 100
        speedup_wp = ((wp_pf / wp_default) - 1) * 100
        speedups_by_prefetcher[pf]["cp"].append(speedup_cp)
        speedups_by_prefetcher[pf]["wp"].append(speedup_wp)

# Create the subplot figure with 2 rows and 1 column
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.3,
    # subplot_titles=["CP Speedup (GAPS, SPEC, LCF)", "WP Speedup (GAPS, LCF)"]
)

# Plot for CP (Top row)
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["cp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
        ),
        row=1,
        col=1,
    )

    fig.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=1,  # First row
        col=1,  # First column
    )

    fig.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=1,  # First row
        col=1,  # First column
    )


# Plot for WP (Bottom row)
for i, (pf, props) in enumerate(prefetchers.items()):
    pattern_shape = pattern_shapes[i % len(pattern_shapes)]
    fig.add_trace(
        go.Bar(
            x=suite_labels,
            y=speedups_by_prefetcher[pf]["wp"],
            name=props["label"],
            marker_color=props["color"],
            marker=dict(pattern=dict(shape=pattern_shape)),
            showlegend=False,  # Hide legend for WP
        ),
        row=2,
        col=1,
    )

    fig.add_vline(
        x=0.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=2,  # First row
        col=1,  # First column
    )

    fig.add_vline(
        x=1.5,
        line_width=2,
        line_dash="dash",
        line_color="gray",
        row=2,  # First row
        col=1,  # First column
    )


# Update layout and styling
fig.update_layout(
    xaxis=dict(
        # title="Suite",
        tickangle=0,
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="Speedup (%) - NO-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-10, 10],
    ),
    yaxis2=dict(
        title="Speedup (%) - ED-WP",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
        range=[-10, 10],
    ),
    barmode="group",
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

fig.add_annotation(
    x=1 + 0.2,
    y=0,
    text=f"{speedups_by_prefetcher['llc-next_line']['cp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
)

fig.add_annotation(
    x=1 + 0.2,
    y=0,
    text=f"{speedups_by_prefetcher['llc-next_line']['wp'][1]:.1f}%",
    showarrow=False,
    font=dict(color="black"),
    yshift=10,
    row=2,
    col=1,
)

# Show the plot
fig.show()

# Save the figure as an interactive HTML file
fig.write_html("llc_prefetchers_plot.html", include_plotlyjs="cdn")

# Save the figure as a static PDF
fig.write_image(
    "llc_prefetchers_plot.pdf", format="pdf", width=1100, height=800, scale=2
)

In [ ]:
import plotly.graph_objects as go

# Only one level: L1I, L1D
levels = ["L1I", "L1D", "L2C", "LLC"]
suites = ["lcf", "gaps", "spec"]

# Initialize containers to store useful and useless diffs for each benchmark
useful_l1i_diffs = []
useless_l1i_diffs = []
useful_l1d_diffs = []
useless_l1d_diffs = []
useful_l2c_diffs = []
useless_l2c_diffs = []
useful_llc_diffs = []
useless_llc_diffs = []
all_benchmarks = []
suite_boundaries = []  # List to store the boundary positions for vertical lines

# Collect data
x_position = 0  # To track the x-axis position for the boundaries

for suite in suites:
    benchmarks = list(
        results[f"{suite}_cp_default_df"].index.drop("gmean", errors="ignore")
    )

    for bm in benchmarks:
        all_benchmarks.append(bm)

        # Collect useful and useless diffs for L1I
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "L1I_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "L1I_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_l1i_diffs.append(useful_diff)
        useless_l1i_diffs.append(useless_diff)

        # Collect useful and useless diffs for L1D
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "L1D_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "L1D_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_l1d_diffs.append(useful_diff)
        useless_l1d_diffs.append(useless_diff)

        # Collect useful and useless diffs for L2C
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "L2C_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "L2C_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_l2c_diffs.append(useful_diff)
        useless_l2c_diffs.append(useless_diff)

        # Collect useful and useless diffs for LLC
        wp_useful = results[f"{suite}_wp_default_df"].at[bm, "LLC_WRONG_PATH_USEFULL"]
        wp_useless = results[f"{suite}_wp_default_df"].at[bm, "LLC_WRONG_PATH_USELESS"]

        useful_diff = (wp_useful / 100_000_000) * 1000
        useless_diff = (wp_useless / 100_000_000) * 1000

        useful_llc_diffs.append(useful_diff)
        useless_llc_diffs.append(useless_diff)

    # Store the boundary position after the last benchmark of the suite
    suite_labels.append((x_position + len(benchmarks) / 2, suite.upper()))
    x_position += len(benchmarks)
    suite_boundaries.append(x_position - 0.5)

# Create x-axis labels for each benchmark
x_labels = all_benchmarks

# Plot
fig = go.Figure()

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

# Colors for the "useful" and "useless" parts of the bar
colors = (
    "steelblue",
    "lightsteelblue",
    "orange",
    "lightcoral",
    "mediumseagreen",
    "lightgreen",
    "mediumorchid",
    "plum",
)

# Create a trace for the "useless" and "useful" values for L1I (swap order)
fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_l1i_diffs,  # Useless comes first (on top)
        name="L1I Useless",
        marker_color=colors[1],  # Color for useless
        legendgroup="L1I Useless",
        offsetgroup="L1I",  # Group L1I bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_l1i_diffs,  # Useful comes second (on bottom)
        name="L1I Useful",
        marker_color=colors[0],  # Color for useful
        legendgroup="L1I Useful",
        offsetgroup="L1I",  # Group L1I bars together
    ),
)

# Second stack for L1D (useless on top and useful on bottom)
fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_l1d_diffs,  # Useless comes first (on top)
        name="L1D Useless",
        marker_color=colors[3],  # Color for L1D Useless
        legendgroup="L1D Useless",
        offsetgroup="L1D",  # Group L1D bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_l1d_diffs,  # Useful comes second (on bottom)
        name="L1D Useful",
        marker_color=colors[2],  # Color for L1D Useful
        legendgroup="L1D Useful",
        offsetgroup="L1D",  # Group L1D bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_l2c_diffs,  # Useless comes first (on top)
        name="L2C Useless",
        marker_color=colors[5],  # Color for L2C Useless
        legendgroup="L2C Useless",
        offsetgroup="L2C",  # Group L2C bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_l2c_diffs,  # Useful comes second (on bottom)
        name="L2C Useful",
        marker_color=colors[4],  # Color for L2C Useful
        legendgroup="L2C Useful",
        offsetgroup="L2C",  # Group L2C bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useless_llc_diffs,  # Useless comes first (on top)
        name="LLC Useless",
        marker_color=colors[7],  # Color for LLC Useless
        legendgroup="LLC Useless",
        offsetgroup="LLC",  # Group LLC bars together
    ),
)

fig.add_trace(
    go.Bar(
        x=x_labels,
        y=useful_llc_diffs,  # Useful comes second (on bottom)
        name="LLC Useful",
        marker_color=colors[6],  # Color for LLC Useful
        legendgroup="LLC Useful",
        offsetgroup="LLC",  # Group LLC bars together
    ),
)

# Update the layout
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
        tickmode="array",
        tickvals=list(range(len(x_labels))),
        ticktext=x_labels,  # Single label per benchmark
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    yaxis=dict(
        title="Relative Change in Prefetch Count (%)",
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
    barmode="relative",  # Ensure the bars are grouped side by side
    legend=dict(
        orientation="h",
        x=0.025,
        y=1.05,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12),
    ),
    plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
)

fig.show()

# Save the figure
# fig.write_html("stacked_per_level_L1I_L1D_prefetch_with_boundaries.html", include_plotlyjs="cdn")
# fig.write_image("stacked_per_level_L1I_L1D_prefetch_with_boundaries.pdf", format="pdf", width=1200, height=400, scale=2)

In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Find the indexes of the amean in all_benchmarks
amean_indexes = [i for i, bm in enumerate(all_benchmarks) if bm == "amean"]

# Filter out the amean indexes from the lists
f_useless_l1i_diffs = [
    diff for i, diff in enumerate(useless_l1i_diffs) if i not in amean_indexes
]
f_useful_l1i_diffs = [
    diff for i, diff in enumerate(useful_l1i_diffs) if i not in amean_indexes
]
f_useless_l1d_diffs = [
    diff for i, diff in enumerate(useless_l1d_diffs) if i not in amean_indexes
]
f_useful_l1d_diffs = [
    diff for i, diff in enumerate(useful_l1d_diffs) if i not in amean_indexes
]
f_useless_l2c_diffs = [
    diff for i, diff in enumerate(useless_l2c_diffs) if i not in amean_indexes
]
f_useful_l2c_diffs = [
    diff for i, diff in enumerate(useful_l2c_diffs) if i not in amean_indexes
]
f_useless_llc_diffs = [
    diff for i, diff in enumerate(useless_llc_diffs) if i not in amean_indexes
]
f_useful_llc_diffs = [
    diff for i, diff in enumerate(useful_llc_diffs) if i not in amean_indexes
]
f_all_benchmarks = [bm for i, bm in enumerate(all_benchmarks) if i not in amean_indexes]

df = pd.DataFrame(
    dict(
        year=f_all_benchmarks,
        var1=f_useless_l1i_diffs,
        var2=f_useful_l1i_diffs,
        var3=f_useless_l1d_diffs,
        var4=f_useful_l1d_diffs,
        var5=f_useless_l2c_diffs,
        var6=f_useful_l2c_diffs,
        var7=f_useless_llc_diffs,
        var8=f_useful_llc_diffs,
    )
)

fig = go.Figure()

# Suite dividers
for boundary in suite_boundaries[:-1]:
    fig.add_vline(x=boundary * 4, line_width=2, line_dash="dash", line_color="gray")

# Suite labels
for xpos, label in suite_labels:
    fig.add_annotation(
        x=xpos,
        y=0.95,
        text=label,
        showarrow=False,
        xref="x",
        yref="paper",
        font=dict(size=18, color="black"),
    )

fig.update_layout(
    # plot_bgcolor="white",
    margin=dict(l=20, r=20, t=30, b=20),
    yaxis=dict(title_text="ED-WP Useful vs Useless Prefetches", showgrid=True),
    barmode="stack",
    legend=dict(
        orientation="h",  # horizontal layout
        yanchor="bottom",
        y=1.02,  # just above the plot area
        xanchor="center",
        x=0.5,
    ),
    xaxis=dict(
        title="Benchmarks",
        tickvals=list(range(len(f_all_benchmarks))),
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor="black",
        zerolinewidth=2,
        griddash="dot",
    ),
)

groups = ["var1", "var2", "var3", "var4", "var5", "var6", "var7", "var8"]
colors = ["blue", "red", "green", "purple", "orange", "pink", "brown", "gray"]
names = [
    "L1I Useless",
    "L1I Useful",
    "L1D Useless",
    "L1D Useful",
    "L2C Useless",
    "L2C Useful",
    "LLC Useless",
    "LLC Useful",
]


i = 0
for r, n, c in zip(groups, names, colors):
    ## put var1 and var2 together on the first subgrouped bar
    if i <= 1:
        fig.add_trace(
            go.Bar(x=[df.year, [" "] * len(df.year)], y=df[r], name=n, marker_color=c),
        )
    ## put var3 and var4 together on the first subgrouped bar
    elif i <= 3 and i > 1:
        fig.add_trace(
            go.Bar(x=[df.year, ["  "] * len(df.year)], y=df[r], name=n, marker_color=c),
        )
    ## put var5 and var6 together on the first subgrouped bar
    elif i <= 5 and i > 3:
        fig.add_trace(
            go.Bar(
                x=[df.year, ["   "] * len(df.year)], y=df[r], name=n, marker_color=c
            ),
        )
    ## put var7 and var8 together on the first subgrouped bar
    elif i <= 7 and i > 5:
        fig.add_trace(
            go.Bar(
                x=[df.year, ["    "] * len(df.year)], y=df[r], name=n, marker_color=c
            ),
        )
    i += 1

fig.show()

# Save the figure as an interactive HTML file
fig.write_html(
    "stacked_per_level_.html",
    include_plotlyjs="cdn",
)

# Save the figure as a static PDF
fig.write_image(
    "stacked_per_level.pdf",
    format="pdf",
    width=1200,
    height=400,
    scale=2,
)

### WIP

In [ ]:
def plot_cp_l1i_prefetcher_speedup_geomean(results):
    suites = ["gaps", "spec", "lcf"]
    prefetchers = {
        "l1i-barsa": {"label": "Barsa", "color": "blue"},
        "l1i-bip": {"label": "BIP", "color": "green"},
        "l1i-djolt": {"label": "Djolt", "color": "orange"},
        "l1i-epi": {"label": "EPI", "color": "purple"},
        "l1i-fnlmma": {"label": "FNL-MMA", "color": "red"},
        "l1i-mana": {"label": "Mana", "color": "magenta"},
        "l1i-next_line": {"label": "Next Line", "color": "yellow"},
        "l1i-pips": {"label": "PIPS", "color": "lime"},
        "l1i-tap": {"label": "TAP", "color": "brown"},
    }

    suite_labels = [s.upper() for s in suites]
    speedups_by_prefetcher = {pf: [] for pf in prefetchers}

    for suite in suites:
        cp_default = results[f"{suite}_cp_default_df"]["IPC"]["gmean"]
        for pf in prefetchers:
            cp_pf = results[f"{suite}_cp_{pf}_df"]["IPC"]["gmean"]
            speedup = ((cp_pf / cp_default) - 1) * 100
            speedups_by_prefetcher[pf].append(speedup)

    fig = go.Figure()

    for pf, props in prefetchers.items():
        fig.add_trace(
            go.Bar(
                x=suite_labels,
                y=speedups_by_prefetcher[pf],
                name=props["label"],
                marker_color=props["color"],
            )
        )

    fig.update_layout(
        xaxis=dict(
            title=dict(text="Suite", standoff=40),
            tickangle=0,
            showgrid=False,
        ),
        yaxis=dict(
            title="Speedup (%)",
            range=[-5, 40],
            showgrid=True,
            gridcolor="lightgray",
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor="black",
            zerolinewidth=2,
            griddash="dot",
        ),
        barmode="group",
        legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
        plot_bgcolor="white",
        margin=dict(l=20, r=20, t=20, b=20),
    )

    fig.show()
    return fig


# Save the plot if needed
fig = plot_cp_l1i_prefetcher_speedup_geomean(results)
# fig.write_image("cp_l1i_prefetchers_speedup.svg", format="svg", width=1100, height=300)

In [ ]:
def plot_cp_l1i_prefetcher_speedup_geomean(results):
    suites = ["gaps", "spec", "lcf"]
    prefetchers = {
        "l1i-barsa": {"label": "Barsa", "color": "blue"},
        "l1i-bip": {"label": "BIP", "color": "green"},
        "l1i-djolt": {"label": "Djolt", "color": "orange"},
        "l1i-epi": {"label": "EPI", "color": "purple"},
        "l1i-fnlmma": {"label": "FNL-MMA", "color": "red"},
        "l1i-mana": {"label": "Mana", "color": "magenta"},
        "l1i-next_line": {"label": "Next Line", "color": "yellow"},
        "l1i-pips": {"label": "PIPS", "color": "lime"},
        "l1i-tap": {"label": "TAP", "color": "brown"},
    }

    suite_labels = [s.upper() for s in suites]
    speedups_by_prefetcher = {pf: [] for pf in prefetchers}

    for suite in suites:
        cp_default = results[f"{suite}_wp_default_df"]["IPC"]["gmean"]
        for pf in prefetchers:
            cp_pf = results[f"{suite}_wp_{pf}_df"]["IPC"]["gmean"]
            speedup = ((cp_pf / cp_default) - 1) * 100
            speedups_by_prefetcher[pf].append(speedup)

    fig = go.Figure()

    for pf, props in prefetchers.items():
        fig.add_trace(
            go.Bar(
                x=suite_labels,
                y=speedups_by_prefetcher[pf],
                name=props["label"],
                marker_color=props["color"],
            )
        )

    fig.update_layout(
        xaxis=dict(
            title=dict(text="Suite", standoff=40),
            tickangle=0,
            showgrid=False,
        ),
        yaxis=dict(
            title="Speedup (%)",
            range=[-5, 40],
            showgrid=True,
            gridcolor="lightgray",
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor="black",
            zerolinewidth=2,
            griddash="dot",
        ),
        barmode="group",
        legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5),
        plot_bgcolor="white",
        margin=dict(l=20, r=20, t=20, b=20),
    )

    fig.show()
    return fig


# Save the plot if needed
fig = plot_cp_l1i_prefetcher_speedup_geomean(results)
# fig.write_image("cp_l1i_prefetchers_speedup.svg", format="svg", width=1100, height=300)

In [ ]:
import plotly.graph_objects as go


def create_speedup_bar_chart(
    results,
    prefetchers,
    benchmarks,
    legend_title="Prefetcher",
    show_default=False,
):
    """
    Creates a grouped bar chart comparing speedup for different prefetchers.
    """
    fig = go.Figure()

    for key, props in prefetchers.items():
        if not show_default and key == "default":
            continue

        # Get the WP and CP configurations for this prefetcher
        wp_key = f"lcf_wp_{key}_df"
        cp_key = f"lcf_cp_{key}_df"

        # Calculate speedup: (WP-<> / CP-<>)
        wp_data = results[wp_key]["IPC"]
        cp_data = results[cp_key]["IPC"]
        speedup = ((wp_data / cp_data) - 1) * 100

        # Drop the "amean" benchmark if it exists
        if "amean" in speedup.index:
            speedup = speedup.drop("amean")

        # Add bar trace for the prefetcher
        fig.add_trace(
            go.Bar(
                x=benchmarks,
                y=speedup,
                name=props["label"],
                marker=dict(
                    color=props.get("color", None),  # Sets the fill color
                    line=dict(color=props.get("edgecolor", None)),  # Sets edge color
                    pattern_shape=props.get("hatch", None),  # Sets hatch pattern
                ),
            )
        )

    fig.update_layout(
        # title=title,
        xaxis=dict(
            title="Benchmark",
            tickangle=-45,
            # tickfont=dict(size=20),
            # titlefont=dict(size=24),
            showgrid=True,  # Show gridlines
            zeroline=True,  # Add a zero line
        ),
        yaxis=dict(
            title="Speedup (%)",
            # tickfont=dict(size=20),
            # titlefont=dict(size=24),
            showgrid=True,  # Show gridlines
            zeroline=True,  # Add a zero line
        ),
        # font=dict(size=24),
        autosize=True,
        height=750,
        # width=1200,
        barmode="group",
        # legend_title=legend_title,
        legend=dict(
            x=0.5,  # Center the legend horizontally
            y=0.15,  # Position the legend above the plot
            xanchor="right",  # Anchor the legend to the center horizontally
            yanchor="top",  # Anchor the legend to the bottom vertically
            orientation="h",  # Horizontal legend
        ),
        # plot_bgcolor="white",  # Remove background color
        # paper_bgcolor="white",  # Remove outer paper background color
        shapes=[
            # Add borders around the plot area (including x and y axes)
            dict(
                type="rect",
                x0=0,
                x1=1,
                y0=0,
                y1=1,
                xref="paper",
                yref="paper",
                line=dict(color="black"),  # Border color and width
            ),
        ],
    )
    return fig


# Define the L1I prefetchers and their properties
l1i_prefetchers = {
    "default": {"label": "FDIP", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l1i-barsa": {"label": "Barsa"},
    "l1i-bip": {"label": "BIP"},
    "l1i-djolt": {"label": "Djolt"},
    "l1i-epi": {"label": "EPI"},
    "l1i-fnlmma": {"label": "FNL-MMA"},
    "l1i-mana": {"label": "Mana"},
    "l1i-next_line": {"label": "Next Line"},
    "l1i-pips": {"label": "PIPS"},
    "l1i-tap": {"label": "TAP"},
}

# Define the benchmarks (these should match the indexes in the data)
benchmarks = results["lcf_wp_default_df"].index

if "amean" in benchmarks:
    benchmarks = benchmarks.drop("amean")

# Plot the speedup for the specified prefetchers
fig = create_speedup_bar_chart(
    results,
    l1i_prefetchers,
    benchmarks,
    # title="L1I Prefetcher Speedup Comparison",
    # legend_title="L1I Prefetcher",
    show_default=True,
)

# Show the plot
fig.show()

# Save the plot as an SVG file
fig.write_image("plot.svg", format="svg", width=1200)

### L1I, L1D, L2C, LLC TODO

### Instruction Footprint

In [ ]:
# Filter out 'gmean' from the DataFrame if it's in the index
cp_default_df_filtered = cp_default_df[cp_default_df.index != "gmean"]

# Convert both instr_foot_print and data_foot_print to KB for the filtered DataFrame
cp_instr_footprint = cp_default_df_filtered["instr_foot_print"] * 64 / 1024
cp_data_footprint = cp_default_df_filtered["data_foot_print"] * 64 / 1024

# Define the positions for the bars
x = np.arange(len(cp_default_df_filtered))  # The x locations for the groups
width = 0.35  # The width of the bars

# Create the plot
plt.figure(figsize=(14, 10))

# Plot both columns
plt.bar(
    x - width / 2,
    cp_instr_footprint,
    width,
    label="Instruction Footprint (KB)",
    color="skyblue",
)
plt.bar(
    x + width / 2, cp_data_footprint, width, label="Data Footprint (KB)", color="salmon"
)

# Set labels and title
plt.xlabel("Benchmark Names")
plt.ylabel("Footprint (KB)")
plt.title("Instruction and Data Footprint per Benchmark")

# Set the x-ticks to be the benchmark names, excluding 'gmean'
plt.xticks(x, cp_default_df_filtered.index, rotation=45, ha="right")

# Add a legend
plt.legend()

# Add grid lines for better readability
plt.grid(linestyle="--", alpha=0.7)

# Save the plot
plt.savefig(
    f"results/{suite}_footprint.svg", dpi=300, bbox_inches="tight", format="svg"
)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Compute instruction footprint in MB for CP and WP
cp_instr_footprint = cp_default_df["instr_foot_print"] * 64 / 1024**2
wp_instr_footprint = wp_default_df["instr_foot_print"] * 64 / 1024**2
wpa_instr_footprint = wpa_default_df["instr_foot_print"] * 64 / 1024**2

# Compute the percentage difference
percentage_diff_wp_cp = (
    (wp_instr_footprint - cp_instr_footprint) / cp_instr_footprint
) * 100

percentage_diff_wpa_cp = (
    (wpa_instr_footprint - cp_instr_footprint) / cp_instr_footprint
) * 100

# Drop gmean
percentage_diff_wp_cp = percentage_diff_wp_cp.drop("gmean")
percentage_diff_wpa_cp = percentage_diff_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=percentage_diff_wp_cp.index,  # Benchmarks on the x-axis
        y=percentage_diff_wp_cp,  # Percentage difference
        name="WP",
        legendgroup="WP",
    )
)

fig.add_trace(
    go.Bar(
        x=percentage_diff_wpa_cp.index,  # Benchmarks on the x-axis
        y=percentage_diff_wpa_cp,  # Percentage difference
        name="WPA",
        legendgroup="WPA",
    )
)

fig.update_layout(
    title="Instruction Footprint Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Instruction Footprint Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### IPC

In [ ]:
# Compute the IPC
cp_ipc = cp_default_df["IPC"]
wp_ipc = wp_default_df["IPC"]
wpa_ipc = wpa_default_df["IPC"]

# Drop amean
cp_ipc = cp_ipc.drop("amean")
wp_ipc = wp_ipc.drop("amean")
wpa_ipc = wpa_ipc.drop("amean")

# Calculate speedup
wp_speedup = ((wp_ipc / cp_ipc) - 1) * 100
wpa_speedup = ((wpa_ipc / cp_ipc) - 1) * 100

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=wp_speedup.index,  # Benchmarks on the x-axis
        y=wp_speedup,  # Speedup
        name="WP",  # Add legend name
        legendgroup="WP",
    )
)

fig.add_trace(
    go.Bar(
        x=wpa_speedup.index,  # Benchmarks on the x-axis
        y=wpa_speedup,  # Speedup
        name="WPA",  # Add legend name
        legendgroup="WPA",
    )
)

# Update layout for better readability
fig.update_layout(
    title="Speedup Due to Wrong Path (Aware) Support",
    yaxis_title="Speedup (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L1I Hits & Misses

In [ ]:
cp_l1i_hits = cp_default_df["L1I_TOTAL_HITS"]
wp_l1i_hits = wp_default_df["L1I_TOTAL_HITS"]
wpa_l1i_hits = wpa_default_df["L1I_TOTAL_HITS"]

cp_l1i_misses = cp_default_df["L1I_TOTAL_MISS"]
wp_l1i_misses = wp_default_df["L1I_TOTAL_MISS"]
wpa_l1i_misses = wpa_default_df["L1I_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l1i_hits_wp_cp = ((wp_l1i_hits - cp_l1i_hits) / cp_l1i_hits) * 100
l1i_misses_wp_cp = ((wp_l1i_misses - cp_l1i_misses) / cp_l1i_misses) * 100

l1i_hits_wpa_cp = ((wpa_l1i_hits - cp_l1i_hits) / cp_l1i_hits) * 100
l1i_misses_wpa_cp = ((wpa_l1i_misses - cp_l1i_misses) / cp_l1i_misses) * 100

# Drop gmean
l1i_hits_wp_cp = l1i_hits_wp_cp.drop("gmean")
l1i_misses_wp_cp = l1i_misses_wp_cp.drop("gmean")

l1i_hits_wpa_cp = l1i_hits_wpa_cp.drop("gmean")
l1i_misses_wpa_cp = l1i_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l1i_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l1i_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1i_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l1i_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l1i_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1i_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L1I Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L1D Hits & Misses

In [ ]:
cp_l1d_hits = cp_default_df["L1D_TOTAL_HITS"]
wp_l1d_hits = wp_default_df["L1D_TOTAL_HITS"]
wpa_l1d_hits = wpa_default_df["L1D_TOTAL_HITS"]

cp_l1d_misses = cp_default_df["L1D_TOTAL_MISS"]
wp_l1d_misses = wp_default_df["L1D_TOTAL_MISS"]
wpa_l1d_misses = wpa_default_df["L1D_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l1d_hits_wp_cp = ((wp_l1d_hits - cp_l1d_hits) / cp_l1d_hits) * 100
l1d_misses_wp_cp = ((wp_l1d_misses - cp_l1d_misses) / cp_l1d_misses) * 100

l1d_hits_wpa_cp = ((wpa_l1d_hits - cp_l1d_hits) / cp_l1d_hits) * 100
l1d_misses_wpa_cp = ((wpa_l1d_misses - cp_l1d_misses) / cp_l1d_misses) * 100

# Drop gmean
l1d_hits_wp_cp = l1d_hits_wp_cp.drop("gmean")
l1d_misses_wp_cp = l1d_misses_wp_cp.drop("gmean")

l1d_hits_wpa_cp = l1d_hits_wpa_cp.drop("gmean")
l1d_misses_wpa_cp = l1d_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l1d_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l1d_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1d_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l1d_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l1d_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l1d_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L1D Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

# Show the plot
fig.show()

### L2C Hits & Misses

In [ ]:
cp_l2c_hits = cp_default_df["L2C_TOTAL_HITS"]
wp_l2c_hits = wp_default_df["L2C_TOTAL_HITS"]
wpa_l2c_hits = wpa_default_df["L2C_TOTAL_HITS"]

cp_l2c_misses = cp_default_df["L2C_TOTAL_MISS"]
wp_l2c_misses = wp_default_df["L2C_TOTAL_MISS"]
wpa_l2c_misses = wpa_default_df["L2C_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
l2c_hits_wp_cp = ((wp_l2c_hits - cp_l2c_hits) / cp_l2c_hits) * 100
l2c_misses_wp_cp = ((wp_l2c_misses - cp_l2c_misses) / cp_l2c_misses) * 100

l2c_hits_wpa_cp = ((wpa_l2c_hits - cp_l2c_hits) / cp_l2c_hits) * 100
l2c_misses_wpa_cp = ((wpa_l2c_misses - cp_l2c_misses) / cp_l2c_misses) * 100

# Drop gmean
l2c_hits_wp_cp = l2c_hits_wp_cp.drop("gmean")
l2c_misses_wp_cp = l2c_misses_wp_cp.drop("gmean")

l2c_hits_wpa_cp = l2c_hits_wpa_cp.drop("gmean")
l2c_misses_wpa_cp = l2c_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=l2c_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=l2c_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=l2c_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=l2c_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=l2c_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=l2c_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="L2C Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    # yaxis=dict(range=[0, 100]),  # Set the y-axis range from 0 to 100
)

# Show the plot
fig.show()

### LLC Hits & Misses

In [ ]:
cp_llc_hits = cp_default_df["LLC_TOTAL_HITS"]
wp_llc_hits = wp_default_df["LLC_TOTAL_HITS"]
wpa_llc_hits = wpa_default_df["LLC_TOTAL_HITS"]

cp_llc_misses = cp_default_df["LLC_TOTAL_MISS"]
wp_llc_misses = wp_default_df["LLC_TOTAL_MISS"]
wpa_llc_misses = wpa_default_df["LLC_TOTAL_MISS"]

# Compute the % change in L1I hits and misses
llc_hits_wp_cp = ((wp_llc_hits - cp_llc_hits) / cp_llc_hits) * 100
llc_misses_wp_cp = ((wp_llc_misses - cp_llc_misses) / cp_llc_misses) * 100

llc_hits_wpa_cp = ((wpa_llc_hits - cp_llc_hits) / cp_llc_hits) * 100
llc_misses_wpa_cp = ((wpa_llc_misses - cp_llc_misses) / cp_llc_misses) * 100

# Drop gmean
llc_hits_wp_cp = llc_hits_wp_cp.drop("gmean")
llc_misses_wp_cp = llc_misses_wp_cp.drop("gmean")

llc_hits_wpa_cp = llc_hits_wpa_cp.drop("gmean")
llc_misses_wpa_cp = llc_misses_wpa_cp.drop("gmean")

# Create the bar chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=llc_hits_wp_cp.index,  # Benchmarks on the x-axis
        y=llc_hits_wp_cp,  # Percentage difference
        name="WP Hits",
        legendgroup="WP Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_hits_wpa_cp.index,  # Benchmarks on the x-axis
        y=llc_hits_wpa_cp,  # Percentage difference
        name="WPA Hits",
        legendgroup="WPA Hits",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_misses_wp_cp.index,  # Benchmarks on the x-axis
        y=llc_misses_wp_cp,  # Percentage difference
        name="WP Misses",
        legendgroup="WP Misses",
    )
)

fig.add_trace(
    go.Bar(
        x=llc_misses_wpa_cp.index,  # Benchmarks on the x-axis
        y=llc_misses_wpa_cp,  # Percentage difference
        name="WPA Misses",
        legendgroup="WPA Misses",
    )
)

fig.update_layout(
    title="LLC Cache Hits and Misses Difference Due to Wrong Path (Aware) Execution",
    yaxis_title="Difference (%)",
    yaxis_tickformat=".2f",  # Format y-axis ticks to 2 decimal places
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    # yaxis=dict(range=[0, 100]),  # Set the y-axis range from 0 to 100
)

# Show the plot
fig.show()

In [ ]:
def all_in_one(cp_df, wp_df, wpa_df, prefetcher_list):
    # Correct Path
    df_cp_l1i_default = cp.group_by(cp_df, "default")
    df_cp_l1i_default = cp.calculate_means(df_cp_l1i_default)

    df_cp_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(cp_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_cp_l1i_prefetchers[key] = cp.calculate_speedup(
                df_cp_l1i_default, prefetcher
            )

    # Wrong Path
    df_wp_l1i_default = cp.group_by(wp_df, "default")
    df_wp_l1i_default = cp.calculate_means(df_wp_l1i_default)

    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(wp_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_wp_l1i_prefetchers[key] = cp.calculate_speedup(
                df_wp_l1i_default, prefetcher
            )

    # Wrong Path Aware
    df_wpa_l1i_default = cp.group_by(wpa_df, "default")
    df_wpa_l1i_default = cp.calculate_means(df_wpa_l1i_default)

    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        if key != "default":
            prefetcher = cp.group_by(wpa_df, key)
            prefetcher = cp.calculate_means(prefetcher)
            df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
                df_wpa_l1i_default, prefetcher
            )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_cp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Correct Path (CP) | Speedup of each instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Wrong Path (WP) | Speedup of each instruction prefetcher compared to WP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Wrong Path Aware (WPA) | Speedup of each instruction prefetcher compared to WPA FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean(
        df_cp_l1i_prefetchers,
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of CP, WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=False,
    ).show()

    # Correct Path
    df_cp_l1i_default = cp.group_by(cp_df, "default")
    df_cp_l1i_default = cp.calculate_means(df_cp_l1i_default)

    # Wrong Path
    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        prefetcher = cp.group_by(wp_df, key)
        prefetcher = cp.calculate_means(prefetcher)
        df_wp_l1i_prefetchers[key] = cp.calculate_speedup(df_cp_l1i_default, prefetcher)

    # Wrong Path Aware
    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        prefetcher = cp.group_by(wpa_df, key)
        prefetcher = cp.calculate_means(prefetcher)
        df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
            df_cp_l1i_default, prefetcher
        )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP FDIP prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean_wp_wpa(
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
    ).show()

    # Wrong Path
    df_wp_l1i_prefetchers = {}
    for key in prefetcher_list:
        cp_prefetcher = cp.group_by(cp_df, key)
        wp_prefetcher = cp.group_by(wp_df, key)

        cp_prefetcher = cp.calculate_means(cp_prefetcher)
        wp_prefetcher = cp.calculate_means(wp_prefetcher)
        df_wp_l1i_prefetchers[key] = cp.calculate_speedup(cp_prefetcher, wp_prefetcher)

    # Wrong Path Aware
    df_wpa_l1i_prefetchers = {}
    for key in prefetcher_list:
        cp_prefetcher = cp.group_by(cp_df, key)
        wpa_prefetcher = cp.group_by(wpa_df, key)

        cp_prefetcher = cp.calculate_means(cp_prefetcher)
        wpa_prefetcher = cp.calculate_means(wpa_prefetcher)
        df_wpa_l1i_prefetchers[key] = cp.calculate_speedup(
            cp_prefetcher, wpa_prefetcher
        )

    # Plotting
    cp.create_bar_plot_per_benchmark(
        df_wp_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WP instruction prefetcher compared to CP respective prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_per_benchmark(
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Speedup of each WPA instruction prefetcher compared to CP respective prefetcher",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
        show_amean=False,
    ).show()

    cp.create_bar_plot_gmean_wp_wpa(
        df_wp_l1i_prefetchers,
        df_wpa_l1i_prefetchers,
        prefetcher_list,
        "Speedup",
        title="Comparison of WP and WPA prefetchers",
        y_axis_title="Speedup (%)",
        legend_title="Prefetcher",
        show_fdip=True,
    ).show()

## L1I

In [ ]:
l1i_prefetchers = {
    "default": {"label": "FDIP", "color": "white", "edgecolor": "black", "hatch": "x"},
    "l1i-barsa": {"label": "Barsa", "color": "blue"},
    "l1i-bip": {"label": "BIP", "color": "green"},
    "l1i-djolt": {"label": "Djolt", "color": "orange"},
    "l1i-epi": {"label": "EPI", "color": "purple"},
    "l1i-fnlmma": {"label": "FNL-MMA", "color": "red"},
    "l1i-fnlmma_new": {"label": "FNL-MMA New", "color": "cyan"},
    "l1i-mana": {"label": "Mana", "color": "magenta"},
    "l1i-next_line": {"label": "Next Line", "color": "yellow"},
    "l1i-pips": {"label": "PIPS", "color": "lime"},
    "l1i-tap": {"label": "TAP", "color": "brown"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l1i_prefetchers)

## L1D

In [ ]:
l1d_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l1d-berti": {"label": "Berti"},
    "l1d-ip_stride": {"label": "IP Stride"},
    "l1d-ipcp": {"label": "IPCP"},
    "l1d-next_line": {"label": "Next Line"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l1d_prefetchers)

## L2C

In [ ]:
l2c_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l2c-ampm": {"label": "AMPM"},
    "l2c-bingo": {"label": "Bingo"},
    "l2c-bop": {"label": "BOP"},
    "l2c-dspatch": {"label": "DSPatch"},
    "l2c-ip_stride": {"label": "IP Stride"},
    "l2c-ipcp": {"label": "IPCP"},
    "l2c-mlop": {"label": "MLOP"},
    "l2c-next_line": {"label": "Next Line"},
    "l2c-sandbox": {"label": "Sandbox"},
    "l2c-scooby": {"label": "Scooby"},
    "l2c-sms": {"label": "SMS"},
    "l2c-spp_dev": {"label": "SPP-Dev"},
    "l2c-spp_ppf_dev": {"label": "SPP-PPF-Dev"},
    "l2c-streamer": {"label": "Streamer"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, l2c_prefetchers)

## LLC

In [ ]:
llc_prefetchers = {
    "default": {"label": "None", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "l2c-ip_stride": {"label": "IP Stride"},
    "l2c-next_line": {"label": "Next Line"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, llc_prefetchers)

## LLC Replacment Policy

In [ ]:
llc_replacement_policies = {
    "default": {"label": "LRU", "color": "white", "edgecolor": "black", "hatch": "\\"},
    "ship": {"label": "SHIP"},
    "srrip": {"label": "SRRIP"},
    "drrip": {"label": "DRRIP"},
    "mockingjay": {"label": "Mockingjay"},
}

In [ ]:
all_in_one(cp_df, wp_df, wpa_df, llc_replacement_policies)